# Functions for metabolic modelling of Arabidopsis thaliana and the liverwort Marchantia polymorpha



created by: Corinna Hartinger (corinna.hartinger@univ.ox.ac.uk / corinna.hartinger1@gmail.com)
project started: October 2020

## Setup functions

In [1]:
def check_constraint_data_file(file_name, sheet_name, time_interval):
    """
    inputs:
        - spreadsheet with time-resolved data on constraints (gas exchange, enzyme activity, light intensity, etc.)
    outputs:
        - data from spreadsheet in a dictonary of lists
        """
    
    constraint_data = pd.read_excel(file_name, sheet_name)
    constraint_dict = {}
 
 #   make a list of all the time phases for which columns exist in the spreadsheet
    all_phases_list = []
    for item in constraint_data.columns:
        if item[0] == "t" and len(item) == 3:
            all_phases_list.append(item)
    constraint_dict["all phases"] = all_phases_list
                
    print("Time phases:\n\t", all_phases_list)
    print("Columns for", str(len(all_phases_list)), "time phases exist in the spreadsheet (" + file_name+", "+sheet_name + ").\n")
#    select the time phases after every time interval (which is defined by user when calling function) 
    phases_list = []
    for i in range(0,len(all_phases_list),time_interval):
        phases_list.append(all_phases_list[i])
    
    print("Chosen time interval is:", time_interval, "hours.")
    print("The selected time phases are:\n\t", phases_list, "\n")
    constraint_dict["selected phases"] = phases_list

    
    #     make a list of all the processes specified in the spreadsheet
    print("Processes found in the spreadsheet: ")
    reactions_list = []
    for row in constraint_data["Process"]:
        if str(row) != "nan":
            reactions_list.append(row)
            print("\t",row)
    constraint_dict["Processes"] = reactions_list
    print("\n")
    

    for ind, row in constraint_data.iterrows():
        temp_constraint_dict = {}        
        temp_constraint_dict["Bounds"] = row["Bounds"]
        
        if type(row["all-total-ratio"]) == str:
            ratios = str(row["all-total-ratio"]).split(",")
            temp_constraint_dict["all_or_ratio"] = ratios
        else:
            temp_constraint_dict["all_or_ratio"] = row["all-total-ratio"]
            
        
        temp_constraint_dict["Type"] = row["Type"]
        
        temp_phase_constraints = []
        for phase in phases_list:
            temp_phase_constraints.append(row[phase])
        temp_constraint_dict["constraints in phases"] = temp_phase_constraints
        
        light_phases = []
        light_phases_light_intensity = []
        dark_phases = []
        if row["Process"] == "Photon_tx":
            for phase in phases_list:
                if row[phase] > 0:
                    light_phases.append(phase[-2:])
                    light_phases_light_intensity.append(row[phase])
                elif row[phase] < 0 :
                    print("ERROR: A negative value was given for Photon influx.")
                else:
                    dark_phases.append(phase[-2:])
            constraint_dict["light phases"]= light_phases
            constraint_dict["light phases intensity"]= light_phases_light_intensity
            constraint_dict["dark phases"]= dark_phases        
        
        constraint_dict[row["Process"]]=temp_constraint_dict
        
        
    return constraint_dict


In [2]:
def constraints_same_across_time_phases(model):
    """modifying and adding reactions in the core model"""
    
    
    ## prohibiting sugar or ammonium uptake, other biomass reactions
    # ie no sucrose or glucose intake, no growth (biomass), and for this species no N uptake in form of ammonium (NH4)

    model.reactions.Sucrose_tx.upper_bound = 0.0
    model.reactions.Sucrose_tx.lower_bound = 0.0
    model.reactions.GLC_tx.upper_bound = 0.0
    model.reactions.GLC_tx.lower_bound = 0.0
    model.reactions.Biomass_tx.upper_bound = 0.0
    model.reactions.Biomass_tx.lower_bound = 0.0
    model.reactions.AraCore_Biomass_tx.upper_bound = 0.0
    model.reactions.AraCore_Biomass_tx.lower_bound = 0.0
    model.reactions.NH4_tx.upper_bound = 0.0
    model.reactions.NH4_tx.lower_bound = 0.0
    
    model.reactions.unlProtHYPO_c.lower_bound = 0.0
    model.reactions.unlProtHYPO_c.upper_bound = 0.0
    
    
    
    ### modifying organic acid vacuolar transport

    # removing the MAL_CIT antiporter reaction (it is unpublished/unconfirmed) 
     #(according to Lee but what about Frei et al., 2018?)

    model.reactions.MAL_CIT_vc.upper_bound = 0
    model.reactions.MAL_CIT_vc.lower_bound = 0
    model.reactions.MAL_CIT_rev_vc.upper_bound = 0
    model.reactions.MAL_CIT_rev_vc.lower_bound = 0
    
    ##changing the old fumarate vacuolar import+export reaction and adding new ones for succinate

        #the old reaction would use protons to import fumarate and use as many to export fumarate
        #Sanu and Josh have rewritten it so that import does not use protons but export requires symport of 2 protons

    model.reactions.FUM_PROTON_vc.remove_from_model()


    #adding the new fumarate vacuolar Import reactions

    fumarate_vacuole_import = Reaction("FUM_PROTON_vc")
    fumarate_vacuole_import.upper_bound = 1000.0
    fumarate_vacuole_import.lower_bound = 0.0
    fumarate_vacuole_import.add_metabolites({model.metabolites.aFUM_v: 0.08, 
                                                  model.metabolites.FUM_c: -1,
                                                  model.metabolites.FUM_v: 0.92,
                                                  model.metabolites.PROTON_v: -0.08})
    model.add_reactions([fumarate_vacuole_import])


    #adding new fumarate vacuolar Export reactions

    fumarate_vac_exp = Reaction("FUM_PROTON_rev_vc")
    fumarate_vac_exp.upper_bound = 1000.0
    fumarate_vac_exp.lower_bound = 0.0 
    fumarate_vac_exp.add_metabolites({model.metabolites.aFUM_v: -0.08, 
                                                  model.metabolites.FUM_c: 1,
                                                  model.metabolites.FUM_v: -0.92,
                                                  model.metabolites.PROTON_v: -1.92,
                                                  model.metabolites.PROTON_c: 2})
    model.add_reactions([fumarate_vac_exp])

    ##adding vacuolar succinate transport
    #adding succinate in the vacuole

    SUC_v = Metabolite("SUC_v", name="SUC_v", compartment="v", charge = -2)
    aSUC_v = Metabolite("aSUC_v", name = "aSUC_v", compartment= "v", charge = -1)
    model.add_metabolites(SUC_v)
    model.add_metabolites(aSUC_v)

    #adding the new succinate vacuolar import reactions
    #pKas from MalvinSketch T: 300, pH vacuole 5.5, pH cytosol 7.6
    succinate_vacuole_import = Reaction("SUC_PROTON_vc")
    succinate_vacuole_import.upper_bound = 1000.0
    succinate_vacuole_import.lower_bound = 0.0 
    succinate_vacuole_import.add_metabolites({model.metabolites.aSUC_v: 0.60, 
                                                  model.metabolites.SUC_c: -1,
                                                  model.metabolites.SUC_v: 0.40,
                                                  model.metabolites.PROTON_v: 0.60})

    model.add_reactions([succinate_vacuole_import])

    #adding new succinate vacuolar export reactions

    succinate_vac_exp = Reaction("SUC_PROTON_rev_vc")
    succinate_vac_exp.upper_bound = 1000.0
    succinate_vac_exp.lower_bound = 0.0 
    succinate_vac_exp.add_metabolites({model.metabolites.aSUC_v: -0.60, 
                                                  model.metabolites.SUC_c: 1,
                                                  model.metabolites.SUC_v: -0.40,
                                                  model.metabolites.PROTON_v: -1.40,
                                                  model.metabolites.PROTON_c: 2})

    model.add_reactions([succinate_vac_exp])
    
    

    
 

In [3]:
def make_biomass_composition(model, file_name, column_name,
                             sheet_name):
    """creates a biomass exchange reaction according to the specified column in a spreadsheet or chooses one of the pre-defined biomass exchange reactions.
    Taken from from SAM_Modelling project, 2nd rotation project Corinna.
    - requirements: import pandas as pd
    - inputs: 
            file with biomass components in column "ID" and coefficients in column specified with column_name"""

    if column_name == "AraCore_Biomass":
        print("Use already existing AraCore_Biomass_tx reaction as objective after multiplying model.")
        cobra_model.reactions.AraCore_Biomass_tx.upper_bound = 1000
        cobra_model.reactions.AraCore_Biomass_tx.lower_bound = -1000
        # need to change this here and for AraCore to have growth rate as constraint on this
    elif column_name == "Biomass":
        print("Use already existing Biomass_tx reaction as objective after multiplying model.")
        cobra_model.reactions.Biomass_tx.upper_bound = 1000
        cobra_model.reactions.Biomass_tx.lower_bound = -1000
    else:
        biomass_composition = pd.read_excel(file_name, sheet_name =sheet_name)

        biomass_dict ={}
        for i in range(len(biomass_composition)):
            current_row = biomass_composition.iloc[i]
            if str(current_row[column_name]) != "nan":
                biomass_dict[current_row["ID"]] = current_row[column_name]

        met_list = []
        for met in model.metabolites:
            met_list.append(met.id)

        biomass_rxn = Reaction(column_name + "_tx")

        biomass_rxn.upper_bound = 1000
        biomass_rxn.lower_bound = -1000

        for item in biomass_dict.keys():
            if item in met_list:
                biomass_rxn.add_metabolites({model.metabolites.get_by_id(item):(-1)*biomass_dict[item]})
            else:
                print(item, "not defined in model")

        model.add_reactions([biomass_rxn])
    
 

In [4]:
def set_up_time_phases(model, time_interval):
    """creates a model with multiple time phases, where each reaction and metabolite exists with a _xx suffix, denoting which phase it belongs to"""

    time_phase_dict= {}
    time_phase_list = []

    models = 0
    for i in range(0,24, time_interval):
        if 24%time_interval != 0:
            print("24hours cannot be divided properly by intervals of " + str(time_interval)+" hours.")
            break

        comp_dict = {}
        temp_model = cobra_model.copy()

        for item in temp_model.compartments:
            comp_dict[item+"_"+(str(i).zfill(2))] = temp_model.compartments[item]+"_"+(str(i).zfill(2))

        temp_model.compartments = comp_dict
        temp_model.compartments.update()

        for met in temp_model.metabolites:
            met.id = str(met.id) + "_"+(str(i).zfill(2))
            met.compartment = str(met.compartment) + "_"+(str(i).zfill(2))
    # the part of updating the name of the compartment of the metabolite is essential to actually update
    # the name compartment dictionary in the model, and the names of the compartments set above have to be the same
    # as the names set here

        for reaction in temp_model.reactions:
            reaction.id = str(reaction.id) + "_"+(str(i).zfill(2))

        exec(f'time_phase_model_{str(i).zfill(2)} = temp_model')
        time_phase_dict[i] = eval("time_phase_model_%s" % (str(i).zfill(2)))
        print("Model created for time phase: "+ str(i))

        models +=1

    if models > 0:
        print(str(models)+" models have been created for each phase of the day with intervals of "+ str(time_interval)+" hours.")
   

    # combining the models for the different time phases to one
    time_phase_model_all = time_phase_dict[0].copy()
    for model_number in time_phase_dict:
        if model_number != 0:
            time_phase_model_all = time_phase_model_all.merge(time_phase_dict[model_number])

    # using this method, metabolites that are not part of any reactions will not exist in all time phases expect for the first
    return time_phase_model_all, time_phase_dict

 

In [5]:
def creating_linker_fluxes(model,constraint_data, time_interval, protons=False, accum_met_list=["GLN_v","VAL_v","PHE_v","TYR_v","L_ALPHA_ALANINE_v","SER_v",
                       "TRP_v","GLT_v","ILE_v","L_ASPARTATE_v","CYS_v","MET_v","bHIS_v",
                       "ARG_v","THR_v","LEU_v","PRO_v","LYS_v","ASN_v","GLY_v", "FUM_v",
                      "SUC_v", "MAL_v", "CIT_v", "STARCH_p", "SUCROSE_v", "GLC_v", "FRU_v", "NITRATE_v"]):
    
    """Adds linker reactions for the metabolites defined in this function.
        Takes into account the charge states for metabolites specified in this function."""

    #as set by Naomi
#     accum_met_list = ["GLN_v","VAL_v","PHE_v","TYR_v","SER_v",
#                        "TRP_v","ILE_v","CYS_v","MET_v","bHIS_v",
#                        "ARG_v","THR_v","LEU_v","PRO_v","LYS_v","ASN_v","GLY_v",
#                        "MAL_v", "CIT_v", "STARCH_p", "SUCROSE_v", "NITRATE_v",PROTON_v]
    
#     accum_met_list = ["GLN_v","VAL_v","PHE_v","TYR_v","L_ALPHA_ALANINE_v","SER_v",
#                        "TRP_v","GLT_v","ILE_v","L_ASPARTATE_v","CYS_v","MET_v","bHIS_v",
#                        "ARG_v","THR_v","LEU_v","PRO_v","LYS_v","ASN_v","GLY_v", "FUM_v",
#                       "SUC_v", "MAL_v", "CIT_v", "STARCH_p", "SUCROSE_v", "GLC_v", "FRU_v", "NITRATE_v", "PROTON_v"]
    
#     #as I have been using it mostly
#     accum_met_list = ["GLN_v","VAL_v","PHE_v","TYR_v","L_ALPHA_ALANINE_v","SER_v",
#                        "TRP_v","GLT_v","ILE_v","L_ASPARTATE_v","CYS_v","MET_v","bHIS_v",
#                        "ARG_v","THR_v","LEU_v","PRO_v","LYS_v","ASN_v","GLY_v", "FUM_v",
#                       "SUC_v", "MAL_v", "CIT_v", "STARCH_p", "SUCROSE_v", "GLC_v", "FRU_v", "NITRATE_v"]
    

    # accum_met_list = ["GLN_v","VAL_v","PHE_v","TYR_v","L_ALPHA_ALANINE_v","SER_v",
    #                    "TRP_v","GLT_v","ILE_v","L_ASPARTATE_v","CYS_v","MET_v","bHIS_v",
    #                    "ARG_v","THR_v","LEU_v","PRO_v","LYS_v","ASN_v","GLY_v", "FUM_v",
    #                   "SUC_v", "MAL_v", "CIT_v", "STARCH_p", "SUCROSE_v", "GLC_v", "FRU_v", "NITRATE_v", "CIS_ACONITATE_v"]

    # aconitate_v_00 = Metabolite("CIS_ACONITATE_v_00", name="CIS_ACONITATE_v_00", compartment="v", charge = -3)
    # aconitate_v_12 = Metabolite("CIS_ACONITATE_v_12", name="CIS_ACONITATE_v_12", compartment="v", charge = -3)

    # model.add_metabolites(aconitate_v_00)
    # model.add_metabolites(aconitate_v_12)
                      
    # ,"HIS_v", # for unknown reason, HIS_v is the only vacuolar amino acid that is not created for all time_phase
    # in the all time phases model, although it is present in the individual time phase models
    amino_acids = ["GLN_v","VAL_v","PHE_v","TYR_v","L_ALPHA_ALANINE_v","SER_v",
                       "TRP_v","GLT_v","ILE_v","L_ASPARTATE_v","CYS_v","MET_v","bHIS_v",
                       "ARG_v","THR_v","LEU_v","PRO_v","LYS_v","ASN_v","GLY_v"]
    metabolites_with_charge_states = ["FUM_v", "SUC_v", "MAL_v", "CIT_v"] 
    #Isocitrate?
    #FUM_v not defined as a transfer metabolite in Naomi's model
    
    time_phases = constraint_data["selected phases"]
    print(time_phases)

    for time_phase in time_phases:
        current_phase = time_phase[1:]
        if int(current_phase) != (24-time_interval):
            next_phase = str(int(time_phase[1:]) + time_interval).zfill(2)
        else:
            next_phase = "00"

        for met in accum_met_list:
            if met in metabolites_with_charge_states:
                temp_linker = Reaction(met+"_linker_"+current_phase+"_to_"+next_phase)
                temp_linker.upper_bound = 1000
                temp_linker.lower_bound = 0
                met_stoich_current_phase = model.reactions.get_by_id(met[:-2]+"_PROTON_vc_"+current_phase).metabolites.get(model.metabolites.get_by_id(met+"_"+current_phase))
                amet_stoich_current_phase = model.reactions.get_by_id(met[:-2]+"_PROTON_vc_"+current_phase).metabolites.get(model.metabolites.get_by_id("a"+met+"_"+current_phase))
                met_stoich_next_phase = model.reactions.get_by_id(met[:-2]+"_PROTON_vc_"+next_phase).metabolites.get(model.metabolites.get_by_id(met+"_"+next_phase))
                amet_stoich_next_phase = model.reactions.get_by_id(met[:-2]+"_PROTON_vc_"+next_phase).metabolites.get(model.metabolites.get_by_id("a"+met+"_"+next_phase))
                temp_linker.add_metabolites({model.metabolites.get_by_id(met+"_"+current_phase):
                                            -met_stoich_current_phase,
                                            model.metabolites.get_by_id("a"+met+"_"+current_phase):
                                             -amet_stoich_current_phase,
                                            model.metabolites.get_by_id(met+"_"+next_phase):
                                            met_stoich_next_phase,
                                            model.metabolites.get_by_id("a"+met+"_"+next_phase):
                                             amet_stoich_next_phase})
#           add protons, released/consumed when charge state changes, optional
                if protons:
                    print("Adding protons to "+met+" linker")
                    temp_linker.add_metabolites({model.metabolites.get_by_id("PROTON_v_"+next_phase):(amet_stoich_current_phase-amet_stoich_next_phase)})
                model.add_reactions([temp_linker])
            else:
                temp_linker = Reaction(met+"_linker_"+current_phase+"_to_"+next_phase)
                temp_linker.upper_bound = 1000
                temp_linker.lower_bound = 0
                temp_linker.add_metabolites({model.metabolites.get_by_id(met+"_"+current_phase):-1,
                                            model.metabolites.get_by_id(met+"_"+next_phase):1})
                model.add_reactions([temp_linker])
        print("Linkers added to time phase", current_phase)
        
 

In [6]:
def changing_charge_states_in_vacuole_rxns(model, file_name, constraint_sheet, ph_sheet, time_interval):
    """coding linker reactions that take into account charge states and modify the import and export reactions, and the linker reaction accordingly"""
    
    constraint_data = pd.read_excel(file_name, sheet_name = constraint_sheet)
    charge_state_data = pd.read_excel(file_name, sheet_name = ph_sheet , index_col = "charge_metabolite")
    
    
#     read which metabolites are in constraint sheet and add the version without a or aa prefix to a list
    metabolites_with_charge_states =[]
    
    for index in charge_state_data.index:
        if str(index)[:1] != "a" and str(index)[:2] != "aa" and str(index) != "nan":
            metabolites_with_charge_states.append(index)
    print("Metabolites for which charge states are specified: ", metabolites_with_charge_states, "\n")
    
#         change the vacuolar transport reactions
    for  item in metabolites_with_charge_states:
        for met in model.metabolites:
            if item == met.id[:-3]:
                print("----", met.id, "---")
                for rxn in met.reactions:
                    phase_number = "t"+met.id[-2:]
                    temp_pH = constraint_data[phase_number][0]
                    print("pH = " + str(temp_pH))

                    if "PROTON_vc_" in rxn.id:
                        fraction_met = round(charge_state_data["pH "+ str(temp_pH)][item] / 100, 2)
                        print(fraction_met)
                        fraction_amet = round(charge_state_data["pH "+ str(temp_pH)]["a"+item] / 100, 2)
                        print(fraction_amet)
#                         remove vacuolar metabolites from _vc reaction
                        rxn.subtract_metabolites({model.metabolites.get_by_id(met.id):rxn.metabolites.get(model.metabolites.get_by_id(met.id)),
                                                 model.metabolites.get_by_id("a"+met.id):rxn.metabolites.get(model.metabolites.get_by_id("a"+met.id)),
                                                 model.metabolites.get_by_id("PROTON_v_"+met.id[-2:]): rxn.metabolites.get(model.metabolites.get_by_id("PROTON_v_"+met.id[-2:]))})
#                         add vacuolar metabolites with new fractions to _vc reaction
                        rxn.add_metabolites({model.metabolites.get_by_id(met.id): fraction_met,
                                            model.metabolites.get_by_id("a"+met.id): fraction_amet,
                                            model.metabolites.get_by_id("PROTON_v_"+met.id[-2:]):fraction_amet})
                        print(rxn)
                        print("\n")
                    elif "PROTON_rev_vc_" in rxn.id:
                        fraction_met = round(charge_state_data["pH "+ str(temp_pH)][item] / 100, 2)
                        print(fraction_met)
                        fraction_amet = round(charge_state_data["pH "+ str(temp_pH)]["a"+item] / 100, 2)
                        print(fraction_amet)
#                         remove vauolar metabolites from rev_vc reaction
                        rxn.subtract_metabolites({model.metabolites.get_by_id(met.id):rxn.metabolites.get(model.metabolites.get_by_id(met.id)),
                                                 model.metabolites.get_by_id("a"+met.id):rxn.metabolites.get(model.metabolites.get_by_id("a"+met.id)),
                                                 model.metabolites.get_by_id("PROTON_v_"+met.id[-2:]): rxn.metabolites.get(model.metabolites.get_by_id("PROTON_v_"+met.id[-2:]))})
#                         add vacuolar metabolites with new fractions to _vc reaction
                        rxn.add_metabolites({model.metabolites.get_by_id(met.id): -fraction_met,
                                            model.metabolites.get_by_id("a"+met.id): -fraction_amet,
                                            model.metabolites.get_by_id("PROTON_v_"+met.id[-2:]):-(2-fraction_amet)})
                        print(rxn)
                        print("\n")
    return metabolites_with_charge_states

 

In [5]:
def light_dark_phases_constraints(model, constraint_data,  
                                  biomass_composition, manual = False, maintenance_even_distribution = True, amino_acid_night_to_day_accum = False):
    """applies constraints that are dependent on which phases have light and which are dark,
    on amino acid accumulation, ATPase maintenance, biomass output, nitrate uptake"""

    light_phases = constraint_data["light phases"]
    light_phases_light_intensity = constraint_data["light phases intensity"]
    if len(light_phases) != len(light_phases_light_intensity):
        print("WARNING!! - There are not as many light intensity values as light phases!")
    dark_phases = constraint_data["dark phases"]
    
    if manual:
        # manual constraints on light to simulate 12h light, 12h dark
        # allows to test model without constraints from spreadsheet
        light_phases_light_intensity = []
        light_intensity_value = 200
        for rxn in model.reactions:
            if rxn.id[:-3] == "Photon_tx" and int(rxn.id[-2:])<12:
                rxn.upper_bound = light_intensity_value
                print(rxn.id, "light")
                light_phases_light_intensity.append(light_intensity_value)
            elif rxn.id[:-3] == "Photon_tx" and int(rxn.id[-2:]) >=12:
                rxn.upper_bound = 0
                print(rxn.id, "dark")
                
        # determining which phases are light phases and which are dark phases
        light_phases =[]
        dark_phases = []
        for rxn in model.reactions:
            if rxn.id[:9] == "Photon_tx":
                if rxn.upper_bound != 0:
                    light_phases.append(rxn.id[-2:])
                else:
                    dark_phases.append(rxn.id[-2:])


    time_phases = light_phases + dark_phases
        
    print("Light phases:",light_phases)
    print("Dark phases:",dark_phases)
    
    ## constraining amino acid accumulation to occur only from light to dark - optional
    last_dark_phase = dark_phases[-1]
    first_light_phase = light_phases[0]

    amino_acid_list = ["GLN_v","VAL_v","PHE_v","TYR_v","L_ALPHA_ALANINE_v","SER_v",
                           "TRP_v","GLT_v","ILE_v","L_ASPARTATE_v","CYS_v","MET_v","bHIS_v",
                           "ARG_v","THR_v","LEU_v","PRO_v","LYS_v","ASN_v","GLY_v"]

    # goes through the linker for amino acids from the last dark phase to the first light phase and constrains it to 0
    if amino_acid_night_to_day_accum == False:
        for aa in amino_acid_list:
            model.reactions.get_by_id(aa+"_linker_"+last_dark_phase+"_to_"+first_light_phase).upper_bound = 0
        
        
    ## implementing constraints on biomass output day and night
    biomass_output_string = biomass_composition+"_tx"

    biomass_total_met = Metabolite("Biomass_output_total")
    biomass_total_met.name = "Biomass_output_total"
    model.add_metabolites(biomass_total_met)

    biomass_light_met = Metabolite("Biomass_output_light")
    biomass_light_met.name = "Biomass_output_light"
    model.add_metabolites(biomass_light_met)

    biomass_light_rxn = Reaction(biomass_output_string + "_light")
    biomass_light_rxn.add_metabolites({model.metabolites.Biomass_output_light: -1,
                                      model.metabolites.Biomass_output_total: 1})
    biomass_light_rxn.upper_bound = 1000
    biomass_light_rxn.lower_bound = -1000
    model.add_reactions([biomass_light_rxn])

    biomass_dark_met = Metabolite("Biomass_output_dark")
    biomass_dark_met.name = "Biomass_output_dark"
    model.add_metabolites(biomass_dark_met)

    biomass_dark_rxn = Reaction(biomass_output_string + "_dark")
    biomass_dark_rxn.add_metabolites({model.metabolites.Biomass_output_dark: -1,
                                      model.metabolites.Biomass_output_total: 1})
    biomass_dark_rxn.upper_bound = 1000
    biomass_dark_rxn.lower_bound = -1000
    model.add_reactions([biomass_dark_rxn])

    biomass_total_rxn = Reaction(biomass_output_string + "_total")
    biomass_total_rxn.add_metabolites({model.metabolites.Biomass_output_total: -1}) 
    biomass_total_rxn.upper_bound = 1000
    biomass_total_rxn.lower_bound = -1000
    model.add_reactions([biomass_total_rxn])

#         creates a reaction that make biomass components from time phase turn into biomass_output_light or 
#          biomass_output_dark, based on value in "Photon_tx"

    for phase in time_phases:
        if phase in light_phases:
            rxn = biomass_output_string+"_"+phase
            model.reactions.get_by_id(rxn).add_metabolites({model.metabolites.Biomass_output_light:1})
        elif phase in dark_phases:
            rxn = biomass_output_string+"_"+phase
            model.reactions.get_by_id(rxn).add_metabolites({model.metabolites.Biomass_output_dark:1})   

#      even distribution of biomass output over light and dark phases, respectively

    for i in range(len(light_phases)):
        temp_pseudo = Metabolite("light_pseudo_biomass_%s" %(light_phases[i]))
        temp_pseudo.name = "light_pseudo_biomass_%s" %(light_phases[i])
        model.add_metabolites([temp_pseudo])
        
        if i != len(light_phases) -1:
            model.reactions.get_by_id(biomass_output_string+"_"+light_phases[i]).add_metabolites({
                temp_pseudo:1})
            model.reactions.get_by_id(biomass_output_string+"_"+light_phases[i+1]).add_metabolites({
                temp_pseudo:-1})
        else:
            print("else")
            model.reactions.get_by_id(biomass_output_string+"_"+light_phases[i]).add_metabolites({
                temp_pseudo:1})
            model.reactions.get_by_id(biomass_output_string+"_"+light_phases[0]).add_metabolites({
                temp_pseudo:-1})
            
    for i in range(len(dark_phases)):
        temp_pseudo = Metabolite("dark_pseudo_biomass_%s" %(dark_phases[i]))
        temp_pseudo.name = "dark_pseudo_biomass_%s" %(dark_phases[i])
        model.add_metabolites([temp_pseudo])
        
        if i != len(light_phases) -1:
            model.reactions.get_by_id(biomass_output_string+"_"+dark_phases[i]).add_metabolites({
                temp_pseudo:1})
            model.reactions.get_by_id(biomass_output_string+"_"+dark_phases[i+1]).add_metabolites({
                temp_pseudo:-1})
        else:
            model.reactions.get_by_id(biomass_output_string+"_"+dark_phases[i]).add_metabolites({
                temp_pseudo:1})
            model.reactions.get_by_id(biomass_output_string+"_"+dark_phases[0]).add_metabolites({
                temp_pseudo:-1})
            
    
    # implementing constraints on nitrate uptake day and night

    nitrate_total_met = Metabolite("Nitrate_total")
    model.add_metabolites(nitrate_total_met)

    nitrate_light_met = Metabolite("Nitrate_light")
    model.add_metabolites(nitrate_light_met)

    nitrate_light_rxn = Reaction("Nitrate_tx_light")
    nitrate_light_rxn.add_metabolites({model.metabolites.Nitrate_light: 1,
                                      model.metabolites.Nitrate_total: -1})
    nitrate_light_rxn.upper_bound = 1000
    nitrate_light_rxn.lower_bound = -1000
    model.add_reactions([nitrate_light_rxn])

    nitrate_dark_met = Metabolite("Nitrate_dark")
    model.add_metabolites(nitrate_dark_met)

    nitrate_dark_rxn = Reaction("Nitrate_tx_dark")
    nitrate_dark_rxn.add_metabolites({model.metabolites.Nitrate_dark: 1,
                                      model.metabolites.Nitrate_total: -1})
    nitrate_dark_rxn.upper_bound = 1000
    nitrate_dark_rxn.lower_bound = -1000
    model.add_reactions([nitrate_dark_rxn])

    nitrate_total_rxn = Reaction("Nitrate_tx_total")
    nitrate_total_rxn.add_metabolites({model.metabolites.Nitrate_total: 1})
    nitrate_total_rxn.upper_bound = 1000
    nitrate_total_rxn.lower_bound = -1000
    model.add_reactions([nitrate_total_rxn])

    for phase in time_phases:
        if phase in light_phases:
            rxn = "Nitrate_tx"+"_"+phase
            model.reactions.get_by_id(rxn).add_metabolites({model.metabolites.Nitrate_light:-1})
        elif phase in dark_phases:
            rxn = "Nitrate_tx"+"_"+phase
            model.reactions.get_by_id(rxn).add_metabolites({model.metabolites.Nitrate_dark:-1})    
    
    ## maintenance costs

    ATP_light = Metabolite("ATP_light")
    ATP_dark = Metabolite("ATP_dark")
    model.add_metabolites([ATP_light, ATP_dark])

    ATP_light_rxn = Reaction("ATPase_tx_light")
    ATP_light_rxn.add_metabolites({ATP_light: -1})
    ATP_dark_rxn = Reaction("ATPase_tx_dark")
    ATP_dark_rxn.add_metabolites({ATP_dark: -1})

    model.add_reactions([ATP_light_rxn, ATP_dark_rxn])
    
    for i in range(len(time_phases)):
        if time_phases[i] in light_phases:
            PPFD = light_phases_light_intensity[i]
            ATPase = (PPFD*0.0049)+2.7851 #PPFD-dependent setup (as in Nadine's and Sanu's models)
            ATPase = 1.854 #PPFD-independent setup (as in Naomi's model)
            print("ATP constraint of",ATPase, "set in time phase", time_phases[i])
            model.reactions.get_by_id("ATPase_tx_"+time_phases[i]).upper_bound = ATPase
            model.reactions.get_by_id("ATPase_tx_"+time_phases[i]).lower_bound = ATPase
            model.reactions.get_by_id("ATPase_tx_"+time_phases[i]).add_metabolites({ATP_light: 1})
        elif time_phases[i] in dark_phases:
            model.reactions.get_by_id("ATPase_tx_"+time_phases[i]).add_metabolites({ATP_dark: 1})
        
#         this constraint sets the cytosolic and mitochondrial NADPH oxidases equal, in light and dark phases
        NADPHox_c_m = model.problem.Constraint(
        (model.reactions.get_by_id("NADPHoxc_tx_"+time_phases[i]).flux_expression)-
              (model.reactions.get_by_id("NADPHoxm_tx_"+time_phases[i]).flux_expression), 
        lb = 0,
        ub= 0,
        name="NADPHox_c_to_m"+time_phases[i])
        model.add_cons_vars(NADPHox_c_m)

#         this constraint sets the cytosolic and plastidial NADPH oxidases equal 
        NADPHox_c_p = model.problem.Constraint(
        (model.reactions.get_by_id("NADPHoxc_tx_"+time_phases[i]).flux_expression)-
              (model.reactions.get_by_id("NADPHoxp_tx_"+time_phases[i]).flux_expression), 
        lb = 0,
        ub= 0,
        name="NADPHox_c_to_p"+time_phases[i])
        model.add_cons_vars(NADPHox_c_p)
        
#      even distribution of ATP maintenance over light and dark phases, respectively
    if maintenance_even_distribution:
    #only applied to the dark phases because ATPase_tx flux is specified dependent on PPFD in light phases
        print("ATP maintenance in the dark was set to be evenly distributed.")
        for i in range(len(dark_phases)):
            temp_pseudo = Metabolite("dark_pseudo_ATP_%s" %(dark_phases[i]))
            temp_pseudo.name = "dark_pseudo_ATP_%s" %(dark_phases[i])
            model.add_metabolites([temp_pseudo])

            if i != len(light_phases) -1:
                model.reactions.get_by_id("ATPase_tx_"+dark_phases[i]).add_metabolites({
                    temp_pseudo:1})
                model.reactions.get_by_id("ATPase_tx_"+dark_phases[i+1]).add_metabolites({
                    temp_pseudo:-1})
            else:
                model.reactions.get_by_id("ATPase_tx_"+dark_phases[i]).add_metabolites({
                    temp_pseudo:1})
                model.reactions.get_by_id("ATPase_tx_"+dark_phases[0]).add_metabolites({
                    temp_pseudo:-1})


In [7]:
def apply_constraints_for_each_phase(model, constraint_data, biomass_composition):

    phases_list = constraint_data["selected phases"]
    reactions_list = constraint_data["Processes"]
    
    print(phases_list)
    print(reactions_list)
    
    for rxn in reactions_list:
        type_of_process = constraint_data[rxn]["Type"]
        bounds = constraint_data[rxn]["Bounds"]
        all_or_ratio = constraint_data[rxn]["all_or_ratio"]
        temp_constraint_list = constraint_data[rxn]["constraints in phases"]

        print("---"+rxn+"---")
        print(temp_constraint_list)
        print(bounds)
        print(type_of_process)
        print(all_or_ratio)
        
        if type_of_process == "direct_reaction":
            print(rxn , "is a Direct Reaction")
            for i in range(len(phases_list)):
                print("------", i, "------")
                if len(temp_constraint_list) != len(phases_list) and len(temp_constraint_list) != 1:
                    print("WARNING!! This reactions does not specify a value for each time phase.")
                else:
                    current_phase_str = phases_list[i][1:]
                    if i != len(phases_list)-1:
                        next_phase_str = phases_list[i+1][1:]
                    else:
                        next_phase_str = phases_list[0][1:]

                    if str(all_or_ratio) == "nan":
                        value = temp_constraint_list[i]
                        if str(value) != "nan":
                            print(value)
                            if bounds == "max":
                                print("Max bounds")
                                if value > 0:
                                    print(value)
                                    print(model.reactions.get_by_id(rxn+"_"+current_phase_str).id)
                                    print(model.reactions.get_by_id(rxn+"_"+current_phase_str).upper_bound)
                                    model.reactions.get_by_id(rxn+"_"+current_phase_str).upper_bound = value
                                    print(value)
                                elif value < 0:
                                    if model.reactions.get_by_id(rxn+"_"+current_phase_str).lower_bound ==0:
                                        print("WARNING! This reaction may be irreversible, and shouldn't be changed.")
                                    model.reactions.get_by_id(rxn+"_"+current_phase_str).lower_bound = value
                                else:
                                    model.reactions.get_by_id(rxn+"_"+current_phase_str).upper_bound = value
#                                 print("Ub of reation", rxn+"_"+current_phase_str, "was set to 0.")
#                                    used to be upper and lower, but problematic for CO2_tx reaction
                            elif bounds == "max+min":
                                print("Max+Min bounds")
                                model.reactions.get_by_id(rxn+"_"+current_phase_str).upper_bound = value
                                model.reactions.get_by_id(rxn+"_"+current_phase_str).lower_bound = value
                                print(rxn, current_phase_str, value)


                            else:
                                print("Bounds are not defined.")
                        else:
                            print("No value was specified for time phase", current_phase_str)
                    elif type(all_or_ratio) == int or type(all_or_ratio) == float:
                        print("One value for all phases specified:", all_or_ratio)
                        value = all_or_ratio
                        if bounds == "max":
                            print("Max bounds")
                            if value > 0:
                                model.reactions.get_by_id(rxn+"_"+current_phase_str).upper_bound = value
                            elif value < 0:
                                if model.reactions.get_by_id(rxn+"_"+current_phase_str).lower_bound ==0:
                                    print("WARNING! This reaction may be irreversible, and shouldn't be changed.")
                                model.reactions.get_by_id(rxn+"_"+current_phase_str).lower_bound = value
                            else:
                                model.reactions.get_by_id(rxn+"_"+current_phase_str).upper_bound = value
#                                 print("Ub of reation", rxn+"_"+current_phase_str, "was set to 0.")
#                                    used to be upper and lower, but problematic for CO2_tx reaction
                        elif bounds == "max+min":
                            print("Max+Min bounds")
                            model.reactions.get_by_id(rxn+"_"+current_phase_str).upper_bound = value
                            model.reactions.get_by_id(rxn+"_"+current_phase_str).lower_bound = value
                            print(rxn, current_phase_str, value)
                        else:
                                print("Bounds are not defined.")
                    else:
                        print("Neither all-total-ratio not the time phase columns contain valid entries.")


        elif type_of_process == "linker":
            print(rxn, "is a Linker reaction.")
            for i in range(len(phases_list)):
                print("------", i, "------")
                if len(temp_constraint_list) != len(phases_list) and len(temp_constraint_list) != 1:
                    print("WARNING!! This reactions does not specify a value for each time phase.")
                else:
                    current_phase_str = phases_list[i][1:]
                    if i != len(phases_list)-1:
                        next_phase_str = phases_list[i+1][1:]
                    else:
                        next_phase_str = phases_list[0][1:]

                    if str(all_or_ratio) == "nan":
                        value = temp_constraint_list[i]
                        if str(value) != "nan":
                            print(value)
                            if bounds == "max":
                                print("Max bounds")
                                if value > 0:
                                    model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).upper_bound = value
                                elif value < 0:
                                    if model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).lower_bound ==0:
                                        print("WARNING! This reaction may be irreversible, and shouldn't be changed.")
                                    model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).lower_bound = value
                                else:
                                    model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).upper_bound = value
                                    model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).lower_bound = value
                                    print("Ub and Lb of reation", rxn+"_"+current_phase_str+"_to_"+next_phase_str, "were set to 0.")
                            elif bounds == "max+min":
                                print("Max+Min bounds")
                                model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).upper_bound = value
                                model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).lower_bound = value
                                print(rxn, current_phase_str, value)
                            elif bounds == "min,max":
                                print("lb=min, ub=max bounds")
                                min_value = float(value.split(";")[0])
                                max_value = float(value.split(";")[1])
                                model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).upper_bound = max_value
                                model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).lower_bound = min_value
                                print(rxn, current_phase_str, "min=",min_value ,"max=",max_value)
                            else:
                                print("Bounds are not defined.")
                    elif type(all_or_ratio) == int or type(all_or_ratio) == float or ":" in all_or_ratio:
                        print("One value for all phases specified:", all_or_ratio)
                        value = all_or_ratio
                        if bounds == "max":
                            print("Max bounds")
                            if value > 0:
                                model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).upper_bound = value
                            elif value < 0:
                                if model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).lower_bound == 0:
                                    print("WARNING! This reaction may be irreversible, and shouldn't be changed.")
                                model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).lower_bound = value
                            else:
                                model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).upper_bound = value
                                model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).lower_bound = value
                                print("Ub and Lb of reation", rxn+"_"+current_phase_str+"_to_"+next_phase_str, "were set to 0.")
                        elif bounds == "max+min":
                            print("Max+Min bounds")
                            model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).upper_bound = value
                            model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).lower_bound = value
                            print(rxn, current_phase_str+"_to_"+next_phase_str, value)
                        elif bounds == "min,max":
                                print("lb=min, ub=max bounds")
                                min_value = float(value.split(";")[0])
                                max_value = float(value.split(";")[1])
                                model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).upper_bound = max_value
                                model.reactions.get_by_id(rxn+"_"+current_phase_str+"_to_"+next_phase_str).lower_bound = min_value
                                print(rxn, current_phase_str, "min=",min_value ,"max=",max_value)
                        else:
                                print("Bounds are not defined.")
                    else:
                        print("Neither all-total-ratio not the time phase columns contain valid entries.")

        elif type_of_process == "indirect_reaction":
            print(rxn, "s an indirect_reaction.")
            if rxn == "growth_rate":
                biomass_output_total_str = biomass_composition +"_tx_total"
                if str(all_or_ratio) != "nan":
                    value = all_or_ratio
                    if bounds == "max":
                        model.reactions.get_by_id(biomass_output_total_str).upper_bound = value
                    elif bounds == "max+min":
                        model.reactions.get_by_id(biomass_output_total_str).upper_bound = value
                        model.reactions.get_by_id(biomass_output_total_str).lower_bound = value
                    else:
                        print("Bounds are not defined.")
                else:
                    print("All-total-ratio does not contain a valid entry.")

                    
        elif type_of_process == "indicator":
            print(rxn, "is an indicator reaction.")
            # temp_pH is implemented in the changing_charge_states_in_vacuole_rxns function
            
            
        elif type_of_process == "multi_reaction":
            for i in range(len(phases_list)):
                if str(all_or_ratio) == "nan" and str(temp_constraint_list[i]) != "nan":
                    value = temp_constraint_list[i]
                elif type(all_or_ratio) == int or type(all_or_ratio) == float and str(all_or_ratio) != "nan":
                    print("One value for all phases specified:", all_or_ratio)
                    value = all_or_ratio
                else:
                    print("Neither all-total-ratio nor constraints in time phases are defined.")
                    break
                    
                if bounds == "max":
                    print("WARNING - only max values note yet implemented for multi_reaction")
                    #if I want to constrain only the maximum value, I'd have to define either the lb or ub, depending on which direction the reaction goes, ie whether the flux is positive or negative
                elif bounds == "max+min":

                    if rxn == "RXN0_5224":
                        native_BCA_const = model.problem.Constraint(
                            (model.reactions.get_by_id(rxn+"_c_"+phases_list[i][1:]).flux_expression+ 
                             model.reactions.get_by_id(rxn+"_m_"+phases_list[i][1:]).flux_expression+
                             model.reactions.get_by_id(rxn+"_p_"+phases_list[i][1:]).flux_expression),
                            lb = value,
                            ub = value)
                        model.add_cons_vars(native_BCA_const)
                        print("Constraint applied to sum of native "+ "RXN0_5224_"+phases_list[i][1:] +" (BCA) _c, _m and _p reactions")
                    elif rxn == "PEPCARBOX_RXN":
                        native_BCA_const = model.problem.Constraint(
                            (model.reactions.get_by_id(rxn+"_c_"+phases_list[i][1:]).flux_expression+ 
                             model.reactions.get_by_id(rxn+"_p_"+phases_list[i][1:]).flux_expression),
                            lb = value,
                            ub = value)
                        model.add_cons_vars(native_BCA_const)
                        print("Constraint applied to sum of native "+ "PEPCARBOX_RXN_"+phases_list[i][1:] +" _c and _p reactions")
                    elif rxn == "MALATE_DEH_RXN":
                        native_BCA_const = model.problem.Constraint(
                            (model.reactions.get_by_id(rxn+"_c_"+phases_list[i][1:]).flux_expression+
                             model.reactions.get_by_id(rxn+"_x_"+phases_list[i][1:]).flux_expression+
                             model.reactions.get_by_id(rxn+"_m_"+phases_list[i][1:]).flux_expression+
                             model.reactions.get_by_id(rxn+"_p_"+phases_list[i][1:]).flux_expression),
                            lb = value,
                            ub = value)
                        model.add_cons_vars(native_BCA_const)     
                        print("Constraint applied to sum of native "+ "MALATE_DEH_RXN_"+phases_list[i][1:] +" _c, _x, _m and _p reactions")
        
        elif type_of_process == "ratio":
            if "nan" not in str(all_or_ratio):
                ratio = [float(all_or_ratio[0]), float(all_or_ratio[1])]
                print(ratio)
                if rxn == "biomass_day-night_ratio":
                    biomass_output_string = biomass_composition+"_tx"
                    print(ratio)
                    biomass_light_dark_const = model.problem.Constraint(
                            ratio[1]*(model.reactions.get_by_id(biomass_output_string + "_light").flux_expression) - 
                        ratio[0]*(model.reactions.get_by_id(biomass_output_string + "_dark").flux_expression), 
                            lb = 0,
                            ub= 0,
                            name="biomass_day_night_const")
                    model.add_cons_vars(biomass_light_dark_const)

                elif rxn == "ATP_to_NADPH_maintenance":
    #             this constraint sets ration between ATPase and the NADPH oxidases in each time phase
                    print("ATP to NADPH maintenance ratio:", all_or_ratio)
                    for i in range(len(phases_list)):
                        ATPase_NADPHox_const = model.problem.Constraint(
                        ratio[1]*(model.reactions.get_by_id("ATPase_tx_"+phases_list[i][1:]).flux_expression) - 
                            ratio[0]*(model.reactions.get_by_id("NADPHoxc_tx_"+phases_list[i][1:]).flux_expression +
                              model.reactions.get_by_id("NADPHoxm_tx_"+phases_list[i][1:]).flux_expression+
                              model.reactions.get_by_id("NADPHoxp_tx_"+phases_list[i][1:]).flux_expression ), 
                        lb = 0,
                        ub= 0,
                        name="ATPase_to_NADPHox_const"+phases_list[i][1:])
                        model.add_cons_vars(ATPase_NADPHox_const)
                elif rxn == "ATP-NADPH-maintenance_day_to_night":
                    # this constraints sets the ratio between ATPase_tx_light the ATPase_tx_dark
                    ATPase_light_dark = model.problem.Constraint(
                            ratio[1]*(model.reactions.get_by_id("ATPase_tx_light").flux_expression) - 
                            ratio[0]*(model.reactions.get_by_id("ATPase_tx_dark").flux_expression), 
                            lb = 0,
                            ub= 0,
                            name="NGAM_day_night_const")
                    model.add_cons_vars(ATPase_light_dark)
                elif rxn == "Nitrate_tx":
                    global nitrate_light_dark_const
                    # this constraint sets the ratio between nitrate uptake during the day and night
                    nitrate_light_dark_const = model.problem.Constraint(
                            ratio[1]*(model.reactions.Nitrate_tx_light.flux_expression) - 
                            ratio[0]*(model.reactions.Nitrate_tx_dark.flux_expression), 
                            lb = 0,
                            ub= 0,
                            name="nitrate_light_dark_const")
                    model.add_cons_vars(nitrate_light_dark_const)
                elif rxn == "Rubisco_C-O":
                    # this constraint sets the ratio between Rubisco carboxylation and oxygenation
                    
                    for i in range(len(phases_list)):
                        print(i, phases_list[i])
                        
                        # only sets a constraint if there is light in that phase
                        if model.reactions.get_by_id("Photon_tx_"+phases_list[i][1:]).upper_bound > 0 :
                        
                            rubisco_c_to_o_const = model.problem.Constraint(
                            ratio[1]*(model.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_"+phases_list[i][1:]).flux_expression) -
                            ratio[0]* model.reactions.get_by_id("RXN_961_p_"+phases_list[i][1:]).flux_expression,
                            lb = 0,
                            ub = 0,
                            name="rubisco_c_to_o_const"+phases_list[i][1:])
                            model.add_cons_vars(rubisco_c_to_o_const)
                            print("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_"+phases_list[i][1:])
                            print("RXN_961_p_"+phases_list[i][1:])
                    print("Rubisco CO constraint:", ratio)
                    
                else:
                    print("ERROR - no constraints defined for this ratio process.")
            else:
                print("Ratio for", rxn, "was not defined.")
            
        else:
            print(rxn, "Type defined in spreadsheet:", type_of_process)
            print("No action defined for this type.")
            

 

In [9]:
def optimise_model(model, objective_reaction, pFBA = True):
    print("Objective:", objective_reaction)
    model.objective = {model.reactions.get_by_id(objective_reaction):1}
    
    if pFBA == True:
        pfba_solution = cobra.flux_analysis.pfba(model)

        return pfba_solution, model
    else:
        solution = model.optimize()
        
        return solution, model

"""    try:
        pfba_solution = cobra.flux_analysis.pfba(model)
        return pfba_solution
    except:
        print("Model infeasible!")"""

'    try:\n        pfba_solution = cobra.flux_analysis.pfba(model)\n        return pfba_solution\n    except:\n        print("Model infeasible!")'

In [10]:
def set_up_time_phase_constrained_model(model, biomass_composition_file, biomass_composition_column):
    
    check_constraint_file()
    
    constraints_same_across_time_phases()
    
    make_biomass_composition()
    
    set_up_time_phases()
    
    apply_constraints_for_each_phase()
    
    light_dark_phases_constraints()
    
    optimise_model()
    
    save_fluxes_by_timephase()
    
    plot_linker_rxn_fluxes()
    
    svg_fluxmap_arrow_modification()

## Analysis functions

In [11]:
#####################################################################
# This function prints information on CO2 generating fluxes and can #
# be used to study dark respiration                                 #
# inputs: 1) an FBA solved model, 2) a list of fluxs to ignore, 3)  #
# the tag used to mark night time metabolites ex: "_night", 4)      #
# a list of solution objects                                        #
#####################################################################
def printDarkRespirationFluxes(model,rxn2avoid=["CO2_tx1","CO2_ec1","CO2_mc1","CO2_pc1","GCVMULTI_RXN_m1"],night_tag = "2",custom_solutions=[]):
  for met in model.metabolites.query("CARBON_DIOXIDE"):
    if met.id.__contains__(night_tag):
      continue
        
    for rxn in met.reactions:
      if rxn2avoid.__contains__(rxn.id):
        continue
      if len(custom_solutions) == 0:
        if (rxn.metabolites.get(met) > 0 and rxn.x > 0) or (rxn.metabolites.get(met) < 0 and rxn.x < 0):
          print(rxn.id+"\t"+rxn.reaction+"\t"+str(rxn.x*rxn.metabolites.get(met)))
        else:
          if (rxn.metabolites.get(met) > 0 and rxn.upper_bound > 0) or (rxn.metabolites.get(met) < 0 and rxn.lower_bound < 0):
            tot=0
            for sol in custom_solutions:
              tot=tot+abs(sol.x_dict.get(rxn.id))
              if tot==0:
                continue

            print(rxn.id+"\t"+rxn.reaction,)
            for sol in custom_solutions:
              print("\t"+str(rxn.metabolites.get(met)*sol.x_dict.get(rxn.id)),)
            print("")

In [12]:
####################################################################
# This function generates CO2 budgets for a given flux distribution #
# inputs: 1) an FBA model, 2) a dictionary object with reaction ids #
# as keys and reaction fluxes as values, 3) name of output file (op-#
# -tional), 4) Option to show plots, 5) If choosing to show plot, c-#
# -hoose wether to use percentage or absolute values in the plot. 6)#
# Provide a day or night indicator tag to specify day or night CO2  #
# summary 7) a destination file to save plot to 8) a dictionary to  #
# specify colour for fluxes in plot                                 #
#####################################################################
def generateCO2budget(model,solution,outfile="",show_plot=True,percentage=False,day_or_night_tag="1",save_plot_to="temp.png",colourDict={}):
  threshold = 0.01
  if outfile!="":
    fout = open(outfile,"w")
  CO2dict = dict()
  total = 0
  for p in ("c","p","m","x","e"):
    met=model.metabolites.get_by_id("CARBON_DIOXIDE_"+p+day_or_night_tag)
    for rxn in met.reactions:
      if rxn.id.__contains__("CO2_mc") or rxn.id.__contains__("CO2_pc") or rxn.id.__contains__("CO2_ec") or rxn.id.__contains__("CO2_xc"):
        continue
      sto=rxn.metabolites.get(met)
      if outfile!="":
        fout.write(rxn.id+"\t"+rxn.reaction+"\t"+str(solution[rxn.id]*(sto))+"\t"+met.compartment+"\n")
      CO2dict[rxn.id]=solution[rxn.id]*(sto)
      if solution[rxn.id]*(sto) > 0:
        total = total + (solution[rxn.id]*(sto))
  print("Total:", total)
  if outfile!="":
    fout.close()

  tempDict = dict()
  for rxn in CO2dict.keys():
    tempDict[rxn]=abs(CO2dict[rxn])

  #sort CO2dict by values
  import operator
  sorted_by_value = sorted(tempDict.items(), key= lambda x:x[1],reverse=True)

  CO2dict2 = dict()
  CO2dict2["Others-pos"]=0
  CO2dict2["Others-neg"]=0
  baseline = dict()
  pos_base=0
  neg_base=0
  i=0
  for TEMP in sorted_by_value:
    rxn = TEMP[0]
    if CO2dict[rxn]>0:
      if CO2dict[rxn] < total*threshold:
        if percentage:
          CO2dict2["Others-pos"]=CO2dict2["Others-pos"]+float(CO2dict[rxn]*100)/total
        else:
          CO2dict2["Others-pos"]=CO2dict2["Others-pos"]+CO2dict[rxn]
        continue
      base = pos_base
      if percentage:
        CO2dict2[rxn]=float(CO2dict[rxn]*100)/total
        pos_base = pos_base + float(CO2dict[rxn]*100)/total
      else:
        pos_base = pos_base + CO2dict[rxn]
        CO2dict2[rxn]=CO2dict[rxn]
    else:
      if abs(CO2dict[rxn]) < total*threshold:
        if percentage:
          CO2dict2["Others-neg"]=CO2dict2["Others-neg"]+float(CO2dict[rxn]*100)/total
        else:
          CO2dict2["Others-neg"]=CO2dict2["Others-neg"]+CO2dict[rxn]
        continue
      base = neg_base
      if percentage:
        CO2dict2[rxn]=float(CO2dict[rxn]*100)/total
        neg_base = neg_base + float(CO2dict[rxn]*100)/total
      else:
        neg_base = neg_base + CO2dict[rxn]
        CO2dict2[rxn]=CO2dict[rxn]
    i=i+1
    baseline[rxn]=base
  baseline["Others-pos"]=pos_base
  baseline["Others-neg"]=neg_base

  if show_plot:
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size': 10}) #sets a global fontsize
    plt.rcParams['xtick.major.size'] = 5 # adjusts tick line length and width
    plt.rcParams['xtick.major.width'] = 1
    plt.rcParams['ytick.major.size'] = 5
    plt.rcParams['ytick.major.width'] = 1
    plt.rcParams['axes.linewidth']=2 # makes axes line thicker
    plt.figure(figsize=(3,4))
    for rxn in CO2dict2.keys():
      if colourDict.keys().__contains__(rxn):
        plt.bar(1,CO2dict2[rxn],width=0.1,bottom=baseline[rxn],label=rxn,color=colourDict[rxn])
      else:
        plt.bar(1,CO2dict2[rxn],width=0.1,bottom=baseline[rxn],label=rxn)
    plt.xlim(0.8,1.2)
    if percentage:
      plt.ylabel("CO2 produced/consumed (%)")
    else:
      plt.ylabel("CO2 produced/consumed (in moles)")
    handles, labels = plt.gca().get_legend_handles_labels()
    labels2=list(set(labels)-set(["Others-neg","Others-pos"]))+list(["Others-neg","Others-pos"])
    handles2=[handles[labels.index(i)] for i in labels2]
    lgd=plt.legend(handles2,labels2,bbox_to_anchor=(1,1))
    plt.axhline(0,linestyle="--",color="black")
    plt.tight_layout
    plt.savefig(save_plot_to, bbox_extra_artists=(lgd,), bbox_inches='tight')



In [13]:
####################################################################
# This function generates ATP budgets for a given flux distribution #
# inputs: 1) an FBA model, 2) a dictionary object with reaction ids #
# as keys and reaction fluxes as values, 3) name of output file (op-#
# -tional), 4) Option to show plots, 5) If choosing to show plot, c-#
# -hoose wether to use percentage or absolute values in the plot. 6)#
# Provide a day or night indicator tag to specify day or night ATP  #
# summary 7) a destination file to save plot to 8) a dictionary to  #
# specify colour for fluxes in plot                                 #
#####################################################################
def generateATPbudget(model,solution,outfile="",show_plot=True,percentage=False,day_or_night_tag="1",save_plot_to="temp.png",colourDict={}):
  if outfile!="":
    fout = open(outfile,"w")
  ATPdict = dict()
  total = 0
  for p in ("c","p","m","x"):
    met=model.metabolites.get_by_id("ATP_"+p+day_or_night_tag)
    met1=model.metabolites.get_by_id("aATP_"+p+day_or_night_tag)
    for rxn in met.reactions:
      if rxn.id.__contains__("ATP_AMP_mc") or rxn.id.__contains__("ATP_ADP_mc") or rxn.id.__contains__("ATP_pc") or rxn.id.__contains__("AMP_ATP_xc") or rxn.id.__contains__("ATP_ADP_Pi_pc"):
        continue
      sto=rxn.metabolites.get(met)
      sto1=rxn.metabolites.get(met1)
      if outfile!="":
        fout.write(rxn.id+"\t"+rxn.reaction+"\t"+str(solution[rxn.id]*(sto+sto1))+"\t"+met.compartment+"\n")
      ATPdict[rxn.id]=solution[rxn.id]*(sto+sto1)
      if solution[rxn.id]*(sto+sto1) > 0:
        total = total + (solution[rxn.id]*(sto+sto1))
  print("Total:", total)
  if outfile!="":
    fout.close()

  tempDict = dict()
  for rxn in ATPdict.keys():
    tempDict[rxn]=abs(ATPdict[rxn])

  #sort ATPdict by values
  import operator
  sorted_by_value = sorted(tempDict.items(), key= lambda x:x[1],reverse=True)

  ATPdict2 = dict()
  ATPdict2["Others-pos"]=0
  ATPdict2["Others-neg"]=0
  baseline = dict()
  pos_base=0
  neg_base=0
  i=0
  for TEMP in sorted_by_value:
    rxn = TEMP[0]
    if ATPdict[rxn]>0:
      if ATPdict[rxn] < total*0.01:
        if percentage:
          ATPdict2["Others-pos"]=ATPdict2["Others-pos"]+float(ATPdict[rxn]*100)/total
        else:
          ATPdict2["Others-pos"]=ATPdict2["Others-pos"]+ATPdict[rxn]
        continue
      base = pos_base
      if percentage:
        ATPdict2[rxn]=float(ATPdict[rxn]*100)/total
        pos_base = pos_base + float(ATPdict[rxn]*100)/total
      else:
        pos_base = pos_base + ATPdict[rxn]
        ATPdict2[rxn]=ATPdict[rxn]
    else:
      if abs(ATPdict[rxn]) < total*0.01:
        if percentage:
          ATPdict2["Others-neg"]=ATPdict2["Others-neg"]+float(ATPdict[rxn]*100)/total
        else:
          ATPdict2["Others-neg"]=ATPdict2["Others-neg"]+ATPdict[rxn]
        continue
      base = neg_base
      if percentage:
        ATPdict2[rxn]=float(ATPdict[rxn]*100)/total
        neg_base = neg_base + float(ATPdict[rxn]*100)/total
      else:
        neg_base = neg_base + ATPdict[rxn]
        ATPdict2[rxn]=ATPdict[rxn]
    i=i+1
    baseline[rxn]=base
  baseline["Others-pos"]=pos_base
  baseline["Others-neg"]=neg_base

  if show_plot:
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size': 10}) #sets a global fontsize
    plt.rcParams['xtick.major.size'] = 5 # adjusts tick line length and width
    plt.rcParams['xtick.major.width'] = 1
    plt.rcParams['ytick.major.size'] = 5
    plt.rcParams['ytick.major.width'] = 1
    plt.rcParams['axes.linewidth']=2 # makes axes line thicker
    plt.figure(figsize=(3,4))
    for rxn in ATPdict2.keys():
      if colourDict.keys().__contains__(rxn):
        plt.bar(1,ATPdict2[rxn],width=0.1,bottom=baseline[rxn],label=rxn,color=colourDict[rxn])
      else:
        plt.bar(1,ATPdict2[rxn],width=0.1,bottom=baseline[rxn],label=rxn)
    plt.xlim(0.8,1.2)
    if percentage:
      plt.ylabel("ATP produced/consumed (%)")
    else:
      plt.ylabel("ATP produced/consumed (in moles)")
    handles, labels = plt.gca().get_legend_handles_labels()
    labels2=list(set(labels)-set(["Others-neg","Others-pos"]))+list(["Others-neg","Others-pos"])
    handles2=[handles[labels.index(i)] for i in labels2]
    lgd=plt.legend(handles2,labels2,bbox_to_anchor=(1,1))
    plt.axhline(0,linestyle="--",color="black")
    plt.tight_layout
    plt.savefig(save_plot_to, bbox_extra_artists=(lgd,), bbox_inches='tight')



In [14]:
#NAD_P_H

####################################################################
# This function generates NADPH budgets for a given flux distribution #
# inputs: 1) an FBA model, 2) a dictionary object with reaction ids #
# as keys and reaction fluxes as values, 3) name of output file (op-#
# -tional), 4) Option to show plots, 5) If choosing to show plot, c-#
# -hoose wether to use percentage or absolute values in the plot. 6)#
# Provide a day or night indicator tag to specify day or night NAD(P)H  #
# summary 7) a destination file to save plot to 8) a dictionary to  #
# specify colour for fluxes in plot                                 #
#####################################################################
def generateNADPHbudget(model,solution,outfile="",show_plot=True,percentage=False,day_or_night_tag="1",save_plot_to="temp.png",colourDict={}):
  if outfile!="":
    fout = open(outfile,"w")
  NADPHdict = dict()
  total = 0
  for p in ("c","p","m","x"):
    met=model.metabolites.get_by_id("NADPH_"+p+day_or_night_tag)
    #met1=model.metabolites.get_by_id("NADH_"+p+day_or_night_tag)
    for rxn in met.reactions:
      sto=rxn.metabolites.get(met)
      #sto1=rxn.metabolites.get(met1)
      if outfile!="":
        fout.write(rxn.id+"\t"+rxn.reaction+"\t"+str(solution[rxn.id]*(sto))+"\t"+met.compartment+"\n")
      NADPHdict[rxn.id]=solution[rxn.id]*(sto)
      if solution[rxn.id]*(sto) > 0:
        total = total + (solution[rxn.id]*(sto))
  print("Total:", total)
  if outfile!="":
    fout.close()

  tempDict = dict()
  for rxn in NADPHdict.keys():
    tempDict[rxn]=abs(NADPHdict[rxn])

  #sort ATPdict by values
  import operator
  sorted_by_value = sorted(tempDict.items(), key= lambda x:x[1],reverse=True)

  NADPHdict2 = dict()
  NADPHdict2["Others-pos"]=0
  NADPHdict2["Others-neg"]=0
  baseline = dict()
  pos_base=0
  neg_base=0
  i=0
  for TEMP in sorted_by_value:
    rxn = TEMP[0]
    if NADPHdict[rxn]>0:
      if NADPHdict[rxn] < total*0.05:
        if percentage:
          NADPHdict2["Others-pos"]=NADPHdict2["Others-pos"]+float(NADPHdict[rxn]*100)/total
        else:
          NADPHdict2["Others-pos"]=NADPHdict2["Others-pos"]+NADPHdict[rxn]
        continue
      base = pos_base
      if percentage:
        NADPHdict2[rxn]=float(NADPHdict[rxn]*100)/total
        pos_base = pos_base + float(NADPHdict[rxn]*100)/total
      else:
        pos_base = pos_base + NADPHdict[rxn]
        NADPHdict2[rxn]=NADPHdict[rxn]
    else:
      if abs(NADPHdict[rxn]) < total*0.05:
        if percentage:
          NADPHdict2["Others-neg"]=NADPHdict2["Others-neg"]+float(NADPHdict[rxn]*100)/total
        else:
          NADPHdict2["Others-neg"]=NADPHdict2["Others-neg"]+NADPHdict[rxn]
        continue
      base = neg_base
      if percentage:
        NADPHdict2[rxn]=float(NADPHdict[rxn]*100)/total
        neg_base = neg_base + float(NADPHdict[rxn]*100)/total
      else:
        neg_base = neg_base + NADPHdict[rxn]
        NADPHdict2[rxn]=NADPHdict[rxn]
    i=i+1
    baseline[rxn]=base
  baseline["Others-pos"]=pos_base
  baseline["Others-neg"]=neg_base

  if show_plot:
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size': 10}) #sets a global fontsize
    plt.rcParams['xtick.major.size'] = 5 # adjusts tick line length and width
    plt.rcParams['xtick.major.width'] = 1
    plt.rcParams['ytick.major.size'] = 5
    plt.rcParams['ytick.major.width'] = 1
    plt.rcParams['axes.linewidth']=2 # makes axes line thicker
    plt.figure(figsize=(3,4))
    for rxn in NADPHdict2.keys():
      if colourDict.keys().__contains__(rxn):
        plt.bar(1,NADPHdict2[rxn],width=0.1,bottom=baseline[rxn],label=rxn,color=colourDict[rxn])
      else:
        plt.bar(1,NADPHdict2[rxn],width=0.1,bottom=baseline[rxn],label=rxn)
    plt.xlim(0.8,1.2)
    if percentage:
      plt.ylabel("NAD(P)H produced/consumed (%)")
    else:
      plt.ylabel("NAD(P)H produced/consumed (in moles)")
    handles, labels = plt.gca().get_legend_handles_labels()
    labels2=list(set(labels)-set(["Others-neg","Others-pos"]))+list(["Others-neg","Others-pos"])
    handles2=[handles[labels.index(i)] for i in labels2]
    lgd=plt.legend(handles2,labels2,bbox_to_anchor=(1,1))
    plt.axhline(0,linestyle="--",color="black")
    plt.tight_layout
    plt.savefig(save_plot_to, bbox_extra_artists=(lgd,), bbox_inches='tight')



In [15]:
#NAD_H

####################################################################
# This function generates NADH budgets for a given flux distribution #
# inputs: 1) an FBA model, 2) a dictionary object with reaction ids #
# as keys and reaction fluxes as values, 3) name of output file (op-#
# -tional), 4) Option to show plots, 5) If choosing to show plot, c-#
# -hoose wether to use percentage or absolute values in the plot. 6)#
# Provide a day or night indicator tag to specify day or night NADH  #
# summary 7) a destination file to save plot to 8) a dictionary to  #
# specify colour for fluxes in plot                                 #
#####################################################################
def generateNADHbudget(model,solution,outfile="",show_plot=True,percentage=False,day_or_night_tag="1",save_plot_to="temp.png",colourDict={}):
  if outfile!="":
    fout = open(outfile,"w")
  NADHdict = dict()
  total = 0
  for p in ("c","p","m","x"):
    met=model.metabolites.get_by_id("NADH_"+p+day_or_night_tag)
    #met1=model.metabolites.get_by_id("NADH_"+p+day_or_night_tag)
    for rxn in met.reactions:
      sto=rxn.metabolites.get(met)
      #sto1=rxn.metabolites.get(met1)
      if outfile!="":
        fout.write(rxn.id+"\t"+rxn.reaction+"\t"+str(solution[rxn.id]*(sto))+"\t"+met.compartment+"\n")
      NADHdict[rxn.id]=solution[rxn.id]*(sto)
      if solution[rxn.id]*(sto) > 0:
        total = total + (solution[rxn.id]*(sto))
  print("Total:", total)
  if outfile!="":
    fout.close()

  tempDict = dict()
  for rxn in NADHdict.keys():
    tempDict[rxn]=abs(NADHdict[rxn])

  #sort ATPdict by values
  import operator
  sorted_by_value = sorted(tempDict.items(), key= lambda x:x[1],reverse=True)

  NADHdict2 = dict()
  NADHdict2["Others-pos"]=0
  NADHdict2["Others-neg"]=0
  baseline = dict()
  pos_base=0
  neg_base=0
  i=0
  for TEMP in sorted_by_value:
    rxn = TEMP[0]
    if NADHdict[rxn]>0:
      if NADHdict[rxn] < total*0.05:
        if percentage:
          NADHdict2["Others-pos"]=NADHdict2["Others-pos"]+float(NADHdict[rxn]*100)/total
        else:
          NADHdict2["Others-pos"]=NADHdict2["Others-pos"]+NADHdict[rxn]
        continue
      base = pos_base
      if percentage:
        NADHdict2[rxn]=float(NADHdict[rxn]*100)/total
        pos_base = pos_base + float(NADHdict[rxn]*100)/total
      else:
        pos_base = pos_base + NADHdict[rxn]
        NADHdict2[rxn]=NADHdict[rxn]
    else:
      if abs(NADHdict[rxn]) < total*0.05:
        if percentage:
          NADHdict2["Others-neg"]=NADHdict2["Others-neg"]+float(NADHdict[rxn]*100)/total
        else:
          NADHdict2["Others-neg"]=NADHdict2["Others-neg"]+NADHdict[rxn]
        continue
      base = neg_base
      if percentage:
        NADHdict2[rxn]=float(NADHdict[rxn]*100)/total
        neg_base = neg_base + float(NADHdict[rxn]*100)/total
      else:
        neg_base = neg_base + NADHdict[rxn]
        NADHdict2[rxn]=NADHdict[rxn]
    i=i+1
    baseline[rxn]=base
  baseline["Others-pos"]=pos_base
  baseline["Others-neg"]=neg_base

  if show_plot:
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size': 10}) #sets a global fontsize
    plt.rcParams['xtick.major.size'] = 5 # adjusts tick line length and width
    plt.rcParams['xtick.major.width'] = 1
    plt.rcParams['ytick.major.size'] = 5
    plt.rcParams['ytick.major.width'] = 1
    plt.rcParams['axes.linewidth']=2 # makes axes line thicker
    plt.figure(figsize=(3,4))
    for rxn in NADHdict2.keys():
      if colourDict.keys().__contains__(rxn):
        plt.bar(1,NADHdict2[rxn],width=0.1,bottom=baseline[rxn],label=rxn,color=colourDict[rxn])
      else:
        plt.bar(1,NADHdict2[rxn],width=0.1,bottom=baseline[rxn],label=rxn)
    plt.xlim(0.8,1.2)
    if percentage:
      plt.ylabel("NADH produced/consumed (%)")
    else:
      plt.ylabel("NADH produced/consumed (in moles)")
    handles, labels = plt.gca().get_legend_handles_labels()
    labels2=list(set(labels)-set(["Others-neg","Others-pos"]))+list(["Others-neg","Others-pos"])
    handles2=[handles[labels.index(i)] for i in labels2]
    lgd=plt.legend(handles2,labels2,bbox_to_anchor=(1,1))
    plt.axhline(0,linestyle="--",color="black")
    plt.tight_layout
    plt.savefig(save_plot_to, bbox_extra_artists=(lgd,), bbox_inches='tight')



In [16]:
#
def cumulativeATP_plot(model,solution,time_phases, objective_reaction, times_growth_rate):
    ATP_consumed_total = 0
    ATPdict = dict()
    for phase in time_phases:
        total = 0
        for p in ("c","p","m","x"):
            met=model.metabolites.get_by_id("ATP_"+p+"_"+phase[1:])
            met1=model.metabolites.get_by_id("aATP_"+p+"_"+phase[1:])
            for rxn in met.reactions:
                if rxn.id.__contains__("ATP_AMP_mc") or rxn.id.__contains__("ATP_ADP_mc") or rxn.id.__contains__("ATP_pc") or rxn.id.__contains__("AMP_ATP_xc") or rxn.id.__contains__("ATP_ADP_Pi_pc"):
                    continue
                sto=rxn.metabolites.get(met)
                sto1=rxn.metabolites.get(met1)

                ATPdict[rxn.id]=solution[rxn.id]*(sto+sto1)
                if solution[rxn.id]*(sto+sto1) > 0:
                    total = total + (solution[rxn.id]*(sto+sto1))
    
    ATP_consumed = 0
    ATP_consumed_list = []
    for phase in time_phases:
        for rxn in ATPdict:
            if rxn[-2:] == phase[1:] and ATPdict[rxn] > 0:
                ATP_consumed += ATPdict[rxn]
        ATP_consumed_list.append(ATP_consumed)

    growth_rate_flux = model.reactions.get_by_id(objective_reaction).flux   
    if times_growth_rate:
        ATP_consumed_times_growth = []

        for item in ATP_consumed_list:
            ATP_consumed_times_growth.append(item*growth_rate_flux)
        plt.plot(time_phases, ATP_consumed_times_growth)
        plt.ylabel("cumulative ATP consumption (umol/m2/s)")
        plt.xlabel("time")
        print("total ATP consumed:", round(ATP_consumed,5))
        print("Growth rate:", growth_rate_flux)

    else:
        plt.plot(time_phases, ATP_consumed_list)
        plt.ylabel("cumulative ATP consumption (umol/m2/s)")
        plt.xlabel("time")
        print("total ATP consumed:", round(ATP_consumed,5))
        
    return ATPdict, total




In [17]:
### from Naomi's my_own_functions.py
def estimateOutputFromNetCO2(model,netCO2uptake,Output_ID="diel_biomass",Vc_ID="RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1",CO2in_ID="CO2_tx1",verbose=False):

    from cobra import flux_analysis
    # Initally constraint Vc flux to net CO2 uptake rate
    print(model.reactions.get_by_id("Photon_tx_00").upper_bound)
    print(model.reactions.get_by_id("ATPase_tx_00").upper_bound)
    model.reactions.get_by_id(Vc_ID).upper_bound = netCO2uptake
    model.reactions.get_by_id(Vc_ID).lower_bound = netCO2uptake
    print(model.reactions.get_by_id(Vc_ID).upper_bound)
    #perform pFBA
    flux_analysis.parsimonious.pfba(model)
    print("1st pFBA optimisation performed")

    #unconstrain Vc
    model.reactions.get_by_id(Vc_ID).lower_bound = 0
    model.reactions.get_by_id(Vc_ID).upper_bound = 1000

    #set loop counter
    i=0

    #Use a while loop to increase Vc flux until net CO2 rate is similar to given value (or loop counter hits 10)
    while((netCO2uptake - model.reactions.get_by_id(CO2in_ID).x)/netCO2uptake > 0.00001 and i<100):
        i=i+1
        prev = model.reactions.get_by_id(Output_ID).x
        # Increment in Vc flux is set by given netCo2 uptake - model predicted CO2 uptake rate in previous pFBA run
        now = prev + (prev*((netCO2uptake - model.reactions.get_by_id(CO2in_ID).x)/netCO2uptake))
        model.reactions.get_by_id(Output_ID).lower_bound = now
        model.reactions.get_by_id(Output_ID).upper_bound = now
        print(now)
        flux_analysis.parsimonious.pfba(model)
        if verbose:
            print("----"+str(i)+"----")
            print("Vc flux ="+str(model.reactions.get_by_id(Vc_ID).x))
            print("net CO2 uptake ="+str(model.reactions.get_by_id(CO2in_ID).x))
            print("Target CO2 uptake ="+str(netCO2uptake))
            print("Before:"+str(prev))
            print("After:"+str(now))
            print("photon flux = "+str(model.reactions.Photon_tx_00.x))
    return prev

In [18]:
def estimateOutputFromNetCO2_Corinna(model,netCO2uptake,Output_ID="Phloem_output_tx_light_dark",
                                      Vc_ID="RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_light",
                                      CO2in_ID="CO2_tx_light",verbose=False, iterations=20, threshold=0.001):
    
    """ SANU'S FUNCTION MODIFIED BY CORINNA
    This function estimates a given output flux at which the net CO2 uptake rate is equal to the user-defined value"""

    print("Photon_tx_light =", model.reactions.get_by_id("Photon_tx_00").upper_bound)
    print("ATPase_tx_light =", model.reactions.get_by_id("ATPase_tx_00").upper_bound)
    
    # Initally constrain Vc flux to user-defined net CO2 uptake rate
    model.reactions.get_by_id(Vc_ID).upper_bound = netCO2uptake
    model.reactions.get_by_id(Vc_ID).lower_bound = netCO2uptake
    
    # unconstrain upper bound of daytime CO2 uptake reaction
    model.reactions.get_by_id(CO2in_ID).upper_bound = 1000 
    
    # define objective reaction
    model.objective = {model.reactions.get_by_id(Output_ID): 1}
    
    #initial model optimisation
    pfba_solution = cobra.flux_analysis.parsimonious.pfba(model)

    #unconstrain Vc
    model.reactions.get_by_id(Vc_ID).lower_bound = 0
    model.reactions.get_by_id(Vc_ID).upper_bound = 1000
    
    print("Starting CO2 uptake =", model.reactions.get_by_id(CO2in_ID).x)
    print("Starting output flux:", Output_ID, "=", model.reactions.get_by_id(Output_ID).x)
    print("\n")

    #set loop counter
    i=0
    #Use a while loop to change Output flux until net CO2 rate is similar to given value 
    #(or loop counter hits limit)
    while abs(model.reactions.get_by_id(CO2in_ID).x - netCO2uptake) >threshold and i<iterations:
        i+=1
        print("----- Iteration "+ str(i)+ "-----")
        
        print("Current difference and ratio:", (model.reactions.get_by_id(CO2in_ID).x - netCO2uptake), (netCO2uptake/model.reactions.get_by_id(CO2in_ID).x ))
        prev = model.reactions.get_by_id(Output_ID).x
        CCE = round((1 +(model.reactions.CO2_tx_12.flux/model.reactions.CO2_tx_00.flux))*100,2)
        print("CCE:", CCE)

        #obtain new bounds for Output flux by multiplying old bounds by 
        #ratio of user-defined C02 intake to current CO2 intake
        
        factor = ((netCO2uptake/model.reactions.get_by_id(CO2in_ID).flux))
        print("Factor:", factor)
        now =  prev * factor

        print("prev:", prev)
        print("now:",now)

        if model.reactions.get_by_id(Output_ID).upper_bound < now:
            model.reactions.get_by_id(Output_ID).upper_bound = now
            model.reactions.get_by_id(Output_ID).lower_bound = now
        else:
            model.reactions.get_by_id(Output_ID).lower_bound = now
            model.reactions.get_by_id(Output_ID).upper_bound = now

        pfba_solution = cobra.flux_analysis.parsimonious.pfba(model)

  

        """#attempt to reduce factor if new bounds turn out to be too high for model to be feasible
        x = 1
        while x <= 5:
            factor = ((netCO2uptake/model.reactions.get_by_id(CO2in_ID).flux)/x)
            print("Factor:", factor)
            now =  prev * factor
            
            try:
                print("prev:", prev)
                print("now:",now)

                if model.reactions.get_by_id(Output_ID).upper_bound < now:
                    model.reactions.get_by_id(Output_ID).upper_bound = now
                    model.reactions.get_by_id(Output_ID).lower_bound = now
                else:
                    model.reactions.get_by_id(Output_ID).lower_bound = now
                    model.reactions.get_by_id(Output_ID).upper_bound = now

                pfba_solution = cobra.flux_analysis.parsimonious.pfba(model)
                break
                
            except:
                print("Error")
                x += 1
                continue
        else:
            print("Not feasible")"""
        

        if verbose:
            print("Vc flux = "+str(model.reactions.get_by_id(Vc_ID).x))
            print("Current CO2 uptake = "+str(model.reactions.get_by_id(CO2in_ID).x))
            print("Target CO2 uptake = "+str(netCO2uptake))
            print("Output_ID before: "+str(prev))
            print("Output_ID after: "+str(now))
            CCE = round((1 +(model.reactions.CO2_tx_12.flux/model.reactions.CO2_tx_00.flux))*100,2)
            print("CCE:", CCE)
            print("Sum of fluxes:", pfba_solution.objective_value)
            print("\n")
    
    else:
        print("Net CO2 uptake value achieved.\n")
        if verbose:
            print("Final results")
            print("----"+str(i)+"----")
            print("Vc flux = "+str(model.reactions.get_by_id(Vc_ID).x))
            print("Current CO2 uptake = "+str(model.reactions.get_by_id(CO2in_ID).x))
            print("Target CO2 uptake = "+str(netCO2uptake))
            print("Output_ID:", model.reactions.get_by_id(Output_ID).flux)
            CCE = round((1 +(model.reactions.CO2_tx_12.flux/model.reactions.CO2_tx_00.flux))*100,2)
            print("CCE:", CCE)
            print("Sum of fluxes:", pfba_solution.objective_value)
            print("\n")
    
    return pfba_solution

        
#     elif model.reactions.get_by_id(CO2in_ID).x - netCO2uptake < 0:
#         print("The model cannot obtain the desired CO2 uptake rate.\n")
#         print("ERROR!\nThe Output rate was not adjusted!")
        
#         return pfba_solution
        
#     elif model.reactions.get_by_id(CO2in_ID).x - netCO2uptake == 0:
#         print("You have already reached the desired CO2 uptake rate.")
#         now = prev
        
    


In [19]:
def estimateOutputFromNetCO2_Corinna_dev(model,netCO2uptake,Output_ID="Phloem_output_tx_light_dark",
                                      Vc_ID="RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_light",
                                      CO2in_ID="CO2_tx_light",verbose=False, iterations=50, threshold=0.001, pFBA=True):
    
    temp_model = model.copy()
    
    """ SANU'S FUNCTION MODIFIED BY CORINNA
    This function estimates a given output flux at which the net CO2 uptake rate is equal to the user-defined value"""
        
#     print("Photon_tx_light =", temp_model.reactions.get_by_id("Photon_tx_00").upper_bound)
#     print("ATPase_tx_light =", temp_model.reactions.get_by_id("ATPase_tx_00").upper_bound)


    # define objective reaction
    temp_model.objective = {temp_model.reactions.get_by_id(Output_ID): 1}
    
    
    #########
    # test whether model can achieve desired CO2 uptake rate
    temp_model.reactions.get_by_id(CO2in_ID).upper_bound = netCO2uptake
    temp_model.reactions.get_by_id(CO2in_ID).lower_bound = netCO2uptake
    
    try:
        if pFBA:
            solution = cobra.flux_analysis.parsimonious.pfba(temp_model)
        else:
            solution = temp_model.optimize() #does not fail with error, just issues solver warning
            # would require some minimisation of difference expt. CO2 and achieved CO2, as is, the model doesn't decrease CO2 flux as objective flux is lowered
        print("Model can achieve desired CO2 rate, without discouraging CO2 refixation.")
        print(temp_model.reactions.get_by_id(Output_ID).x)
    except:
        print("\033[91m"+"\nModel cannot achieve desired CO2 rate, even without discouraging CO2 refixation!\033[00m")
        # exit function
        raise Exception("\033[91m"+"\nModel cannot achieve desired CO2 rate, even without discouraging CO2 refixation!\033[00m")
    ######
    
    # unconstrain upper bound of daytime CO2 uptake reaction
    temp_model.reactions.get_by_id(CO2in_ID).upper_bound = 1000
    temp_model.reactions.get_by_id(CO2in_ID).lower_bound = -1000 
    
    #initial model optimisation
    try:
        if pFBA:
            solution = cobra.flux_analysis.parsimonious.pfba(temp_model)
        else:
            solution = temp_model.optimize()
    except:
        print("\033[91m"+"\nFirst optimisation: Model is infeasible!\033[00m")
        return
    else:
        print("First optimisation: Model is feasible.")
        current_CO2 = temp_model.reactions.get_by_id(CO2in_ID).x
        current_output = temp_model.reactions.get_by_id(Output_ID).x

    print("Starting CO2 uptake =", temp_model.reactions.get_by_id(CO2in_ID).x)
    print("Starting output flux:", Output_ID, "=", temp_model.reactions.get_by_id(Output_ID).x)
    print("\n")

    
    #Use a while loop to change Output flux until net CO2 rate is similar to given value 
    #(or loop counter hits limit)
    i=0
    while abs(temp_model.reactions.get_by_id(CO2in_ID).x - netCO2uptake) > threshold and i<iterations:
        i+=1
        print("---- CO2 adjustment iteration "+ str(i)+ " ----")
        
        prev = temp_model.reactions.get_by_id(Output_ID).x
        CCE = round((1 +(temp_model.reactions.CO2_tx_12.flux/temp_model.reactions.CO2_tx_00.flux))*100,2)
        print("CCE:", CCE)

        #obtain new bounds for Output flux by multiplying old bounds by 
        #ratio of user-defined C02 intake to current CO2 intake
        
        diff = temp_model.reactions.get_by_id(CO2in_ID).x - netCO2uptake
        factor = (netCO2uptake/temp_model.reactions.get_by_id(CO2in_ID).flux)
        print("Factor:", round(factor, 4))
        
        now =  prev * factor
#         now = prev + (prev*(netCO2uptake - temp_model.reactions.get_by_id(CO2in_ID).flux/netCO2uptake))

        print("prev:", prev)
        print("now:",now)

        if temp_model.reactions.get_by_id(Output_ID).upper_bound < now:
            temp_model.reactions.get_by_id(Output_ID).upper_bound = now
            temp_model.reactions.get_by_id(Output_ID).lower_bound = now
        else:
            temp_model.reactions.get_by_id(Output_ID).lower_bound = now
            temp_model.reactions.get_by_id(Output_ID).upper_bound = now
            
        try:
            if pFBA:
                solution = cobra.flux_analysis.parsimonious.pfba(temp_model)
            else:
                solution = temp_model.optimize()
            current_output = now
        except:
            raise Exception("Iteration infeasible (output bounds ="+ str(now)+")")
            current_output = prev

    
        

        if verbose:
            print("Vc flux = "+str(temp_model.reactions.get_by_id(Vc_ID).x))
            print("Current CO2 uptake = "+str(temp_model.reactions.get_by_id(CO2in_ID).x))
            print("Target CO2 uptake = "+str(netCO2uptake))
            print("Output_ID before: "+str(prev))
            print("Output_ID after: "+str(now))
            CCE = round((1 +(temp_model.reactions.CO2_tx_12.flux/temp_model.reactions.CO2_tx_00.flux))*100,2)
            print("CCE:", CCE)
            print("Sum of fluxes:", solution.objective_value)
            print("\n")
    
    else:
        if verbose:
            print("Final results")
            print("----"+str(i)+"----")
            print("Vc flux = "+str(temp_model.reactions.get_by_id(Vc_ID).x))
            print("Current CO2 uptake = "+str(temp_model.reactions.get_by_id(CO2in_ID).x))
            print("Target CO2 uptake = "+str(netCO2uptake))
            print("Output_ID:", temp_model.reactions.get_by_id(Output_ID).flux)
            CCE = round((1 +(temp_model.reactions.CO2_tx_12.flux/temp_model.reactions.CO2_tx_00.flux))*100,2)
            print("CCE:", CCE)
            print("Sum of fluxes:", solution.objective_value)
            print("\n")
            
        if i < iterations:
            print("Net CO2 uptake value achieved.\n")
        else:
            raise Exception("\033[91m"+"\nMaximum iterations reached: Desired CO2 uptake value not reached after "+str(i)+" iterations.\033[00m")
        
        
    
    return solution, temp_model


In [3]:
def model_results_overview(model, solut, rounding_nr = 5):
    process_flux_dict ={}
    list_of_linkers = []
    

    for rxn in model.reactions:
        if "linker" in rxn.id and rxn.id[:-9] not in list_of_linkers:
            list_of_linkers.append(rxn.id[:-9])
            
    for linker in list_of_linkers:
        current_linker_fluxes = []
        for rxn in model.reactions:
            if rxn.id[:-9] == linker:
                current_linker_fluxes.append(round(rxn.flux, rounding_nr))
        process_flux_dict[linker] = current_linker_fluxes

    print("Photon uptake =", round(model.reactions.Photon_tx_00.flux,4), "  ", "% of allowed Photon uptake =", round((model.reactions.Photon_tx_00.flux/model.reactions.Photon_tx_00.upper_bound)*100,2))
    print("growth rate =", round(model.reactions.get_by_id(objective_reaction).flux,4), "\t\tsum of fluxes:" ,round(solut.objective_value, 4))
    print("gas exchange =", "Day:",round(model.reactions.CO2_tx_00.flux, 5), "\tNight:",round(model.reactions.CO2_tx_12.flux, 5))
    CCE = 0
    if model.reactions.CO2_tx_12.flux <=0:
        CCE = round((1 +(model.reactions.CO2_tx_12.flux/model.reactions.CO2_tx_00.flux))*100,2)
        print("CCE:", CCE)
    else:
        print("CO2_tx at night is taking up CO2.")
    print("ATPase: ", round(model.reactions.ATPase_tx_00.flux,4), round(model.reactions.ATPase_tx_12.flux,4))
    print("Rubisco Carbox./Oxygen. = ", round(model.reactions.RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_00.flux, 4),"/", round(model.reactions.RXN_961_p_00.flux, 4))
    print("Linker fluxes \t\t Day \t Night")
    for process in process_flux_dict:
#         print(process, "\t\t",process_flux_dict[process][0],"\t",process_flux_dict[process][1])
        if round(process_flux_dict[process][0],rounding_nr) == round(model.reactions.get_by_id(process+"_00_to_12").upper_bound, rounding_nr):
            day_flux = "\033[91m"+str(round(process_flux_dict[process][0]))+"\033[00m"
        else:
            day_flux = str(round(process_flux_dict[process][0], rounding_nr))
            
        if round(process_flux_dict[process][1],rounding_nr) == round(model.reactions.get_by_id(process+"_12_to_00").upper_bound, rounding_nr):
            night_flux = "\033[91m"+str(round(process_flux_dict[process][1], rounding_nr))+"\033[00m"
        else:
            night_flux = str(round(process_flux_dict[process][1], rounding_nr))

        print('{:<20}{:>20}{:>20}'.format(process, day_flux, night_flux))
        


In [2]:
def apply_sets_of_constraints(input_model, constraints_list, mode, unbound_objective=True, verbose=False, iterations=5):
    """
    
    """
    # mode = "cumulative", "combination" or "variation"
    assert mode in ["cumulative", "combination", "variation"], 'mode parameter must be "cumulative", "combination" or "variation"'
    
    metrics_list = ["noct. CO2 coeff.", "CCE", "sum of fluxes", "photon use", "growth rate ub", "photon use ub", "daytime CO2 uptake ub", "total aa day", "total aa night", "feasibility"]
    print("len(metrics_list) = ", len(metrics_list))
    ### sets reference model
    model = input_model.copy()
    

    if unbound_objective:
        model.reactions.get_by_id(objective_reaction).upper_bound = 1000

    # creates a dictionary containing as many copies of the reference model as needed to compute all different combinations of the constraints
    model_dict = {}
    if mode == "variation":
        model_iterations = iterations +1
    elif mode == "combination":
        model_iterations = len(constraints_list)**2
    elif mode == "cumulative":
        model_iterations = len(constraints_list)+1
        
    for i in range(model_iterations):
        exec(f'model_{str(i).zfill(2)} = model.copy()')
        model_dict["model_"+str(i).zfill(2)] = eval(f'model_{str(i).zfill(2)}')

    # creates a dictionary to set whether constraint is applied in iteration or not
    condition_dict = {}
    for count, item in enumerate(constraints_list):
        condition_dict[item] = False

    constraints_str_list = [str(item.__name__) for item in constraints_list]
    constraints_str_list.append("feasibility")
    constraints = {"Constraint": constraints_str_list}
    df_constraints = pd.DataFrame(constraints)


    solution_dict = {}

    # gets all reaction ids from the reference model
    rxn_list = []
    for rxn in model.reactions:
        rxn_list.append(rxn.id)

    # add other metrics to save in dataframe
    for metric in metrics_list:
        rxn_list.append(metric)

    # create dataframe (rows: reactions and other metrics; columns: flux/value for each model iteration with the different constraint combinations)
    model_rxns = {"reactions": rxn_list}
    df = pd.DataFrame(model_rxns)


    # iterate over models and apply the constraints such that every iteration has one more constraint applied than the one before
    # saves the results in a dictionary with all solutions and a dataframe with all fluxes and values for metrics
        
    for iteration, md in enumerate(model_dict):
        global estimate_Output
        estimate_Output = False
        
        global random_CO2
        random_CO2 = False
         

        fluxes_list = []
        const_list = []

        model = model_dict[md]
        print("----", iteration+1, "----")        

        if mode == "cumulative":
            # switches each constraint on sequentially
            for count, item in enumerate(condition_dict):
                if count+1 == iteration:
                    condition_dict[item] = True
        elif mode == "combination":
            # switches each individual constraint on or off, to achieve all possible combinations
            for count, item in enumerate(condition_dict):
                switch_number = 2**len(constraints_list)/(2**(count+1))
                if iteration % switch_number == 0:
                    condition_dict[item] = not condition_dict[item]
        elif mode == "variation":
            # creates one "standard" model with the currently defined constraints and then creates x iterations of the model and applies all of the specified functions to vary specific parameters to the each model iteration
            for x in range(iterations+1):
                # after the first iteration, set all conditions to true
                if iteration > 0:
                    for item in condition_dict:
                        condition_dict[item] = True

        for item in condition_dict:
            if condition_dict[item]:
                if mode == "variation":
                    value = item(model)
                    print("Applying constraint:", str(item.__name__))
                    const_list.append(value)
                else:
                    item(model)
                    print("Applying constraint:", str(item.__name__))
                    const_list.append("+")
            else:
                const_list.append("-")

        # try to optimise the model
        try:

            # optimises the model either with estimateOutput function (with pFBA) or regular pFBA optimisation
            if estimate_Output == True:
                if random_CO2:
                    temp_sol, temp_model = estimateOutputFromNetCO2_Corinna_dev(model,netCO2uptake=random_CO2_rate, Output_ID=objective_reaction,
                                                  Vc_ID="RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_00",
                                                  CO2in_ID="CO2_tx_00",verbose=verbose, iterations=30, threshold = 0.0001)
                else:
                    temp_sol, temp_model = estimateOutputFromNetCO2_Corinna_dev(model,netCO2uptake=expt_CO2_assim, Output_ID=objective_reaction,
                                                  Vc_ID="RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_00",
                                                  CO2in_ID="CO2_tx_00",verbose=verbose, iterations=30, threshold = 0.0001)


            else:
                temp_sol, temp_model = optimise_model(model, objective_reaction)

        except:
                # if infeasible, save every flux and metric as NaN to add to dataframe
                for rxn in model.reactions:
                    fluxes_list.append(np.nan)
                print("len(fluxes_list) = ", len(fluxes_list))

                for metric in metrics_list:
                    if "metric" != "feasibility":
                        fluxes_list.append(np.nan)
                    else:
                        fluxes_list.append(int(0))

                print("\033[91m"+"Model iteration is infeasible!\033[00m")
                
                const_list.append(int(0))

        else:
        # runs when model optimisation produced feasible result
        # saves fluxes of each reactions, and value of metric to add to dataframe


            print("Model iteration feasible. Objective flux:", round(temp_model.reactions.get_by_id(objective_reaction).flux,7))
            for rxn in temp_model.reactions:
                flux = rxn.flux
                if "linker_12_to_00" in rxn.id:
                    flux = (-1)*flux #to make night to day accumulation negative for plotting later

                fluxes_list.append(flux)

            #noct. CO2 coeff.
            CO2_coeff = calculate_noct_CO2_refixation_coefficient(temp_model, verbose=True)
            fluxes_list.append(CO2_coeff)

            #CCE
            CCE = calculate_CCE(temp_model)
            fluxes_list.append(CCE)

            #calculating sum of all fluxes
            sum_of_fluxes = calculate_sum_of_all_fluxes(temp_sol)
            fluxes_list.append(sum_of_fluxes)

            #calculating photon use
            fluxes_list.append(temp_model.reactions.Photon_tx_00.flux/temp_model.reactions.Photon_tx_00.upper_bound)

            #objective, photon, CO2 upper bound
            fluxes_list.append(temp_model.reactions.get_by_id(objective_reaction).upper_bound)
            fluxes_list.append(temp_model.reactions.Photon_tx_00.upper_bound)
            fluxes_list.append(temp_model.reactions.CO2_tx_00.upper_bound)

            #total amino acid accumulation
            amino_acids = ["GLN_v","VAL_v","PHE_v","TYR_v","L_ALPHA_ALANINE_v","SER_v",
                       "TRP_v","GLT_v","ILE_v","L_ASPARTATE_v","CYS_v","MET_v","bHIS_v",
                       "ARG_v","THR_v","LEU_v","PRO_v","LYS_v","ASN_v","GLY_v"]
            aa_sum_day = 0
            aa_sum_night = 0
            for item in amino_acids:
                for rxn in temp_model.reactions:
                    if item in rxn.id and "linker_00_to_12" in rxn.id:
                        aa_sum_day += rxn.flux
                    elif item in rxn.id and "linker_12_to_00" in rxn.id:
                        aa_sum_night += rxn.flux
            fluxes_list.append(aa_sum_day)
            fluxes_list.append((-1)*aa_sum_night)

            #save whether model was feasible or not
            fluxes_list.append(int(1)) #1 means feasible (which I decided to use rather than a string)
            const_list.append(int(1))
            print("len(fluxes_list) = ", len(fluxes_list))


            
            # save current solution to solution_dict
            solution_dict[md]= temp_sol


        #name column in dataframe after current model and insert fluxes/metric values
        column_name = md
        df.insert(iteration+1, column_name, fluxes_list)

        df_constraints.insert(iteration+1, column_name, const_list)
        print(const_list)

    df.set_index("reactions", inplace= True)
    df_constraints.set_index("Constraint", inplace= True)
    
    return df, df_constraints


In [6]:
def apply_sets_of_constraints2(input_model, constraints_dict, mode, unbound_objective=True, verbose=False, iterations=5, estimate_Output = False):
    """
    
    """
    
    
    metrics_list = ["noct. CO2 coeff.", "CCE", "sum of fluxes", "photon use", "growth rate ub", "photon use ub", "daytime CO2 uptake ub", "total aa day", "total aa night", "feasibility"]
    print("len(metrics_list) = ", len(metrics_list))
    ### sets reference model
    model = input_model.copy()
    

    if unbound_objective:
        model.reactions.get_by_id(objective_reaction).upper_bound = 1000

    # creates a dictionary containing as many copies of the reference model as needed to compute all different combinations of the constraints
    model_iterations = find_necess_nr_of_models(mode, len(constraints_dict["parameter_functions"]), iterations)
        
    model_dict = create_x_model_objects(model, model_iterations)

    # creates a dictionary to set whether constraint is applied in iteration or not
    condition_dict = {}
    for count, item in enumerate(constraints_dict["parameter_functions"]):
        condition_dict[item] = False

    ### this modifies the original constraints_dict
    # constraints = {"Constraint": constraints_dict["parameter_names"]}.copy()
    # # constraints["Constraint"].append("estimate_output")
    # constraints["Constraint"].append("feasibility")
    
    ##
    constraints = dict()
    temp_con_list = []
    for item in constraints_dict["parameter_names"]:
        temp_con_list.append(item)
    temp_con_list.append("feasibility")
    temp_con_list.append("nCO2RC")
    constraints["Constraint"] = temp_con_list
    
    
    print(constraints)
    df_constraints = pd.DataFrame(constraints)


    solution_dict = {}

    # gets all reaction ids from the reference model
    rxn_list = []
    for rxn in model.reactions:
        rxn_list.append(rxn.id)

    # add other metrics to save in dataframe
    for metric in metrics_list:
        rxn_list.append(metric)

    # create dataframe (rows: reactions and other metrics; columns: flux/value for each model iteration with the different constraint combinations)
    model_rxns = {"reactions": rxn_list}
    df = pd.DataFrame(model_rxns)


    # iterate over models and apply the constraints such that every iteration has one more constraint applied than the one before
    # saves the results in a dictionary with all solutions and a dataframe with all fluxes and values for metrics
        
    for iteration, md in enumerate(model_dict):        
        global random_CO2
        random_CO2 = True
         

        fluxes_list = []
        const_list = []

        model = model_dict[md]
        print("----", iteration+1, "----")        
        
        if iteration != 0:
            #apply constraints for sensitivity analysis
            for (func, sob) in zip(constraints_dict["parameter_functions"], constraints_dict["sobol_sequences"][iteration]):
                lb , _ = func(model, sob)
                if func.__name__ == "set_CO2_tx_day":
                    print("setting random CO2:", lb)
                    random_CO2_rate = lb
                print("Applying constraint:", str(func.__name__),":", round(lb,5))
                const_list.append(lb)
                
        else:
            for i in range(len(constraints_dict["parameter_functions"])):
                const_list.append("-")

        # try to optimise the model
        try:

            # optimises the model either with estimateOutput function (with pFBA) or regular pFBA optimisation
            if estimate_Output == True:
                if random_CO2:
                    temp_sol, temp_model = estimateOutputFromNetCO2_Corinna_dev(model,netCO2uptake=random_CO2_rate, Output_ID=objective_reaction,
                                                  Vc_ID="RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_00",
                                                  CO2in_ID="CO2_tx_00",verbose=verbose, iterations=80, threshold = 0.0001)
                else:
                    temp_sol, temp_model = estimateOutputFromNetCO2_Corinna_dev(model,netCO2uptake=expt_CO2_assim, Output_ID=objective_reaction,
                                                  Vc_ID="RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_00",
                                                  CO2in_ID="CO2_tx_00",verbose=verbose, iterations=80, threshold = 0.0001)


            else:
                temp_sol, temp_model = optimise_model(model, objective_reaction)

        except:
                # if infeasible, save every flux and metric as NaN to add to dataframe
                for rxn in model.reactions:
                    fluxes_list.append(np.nan)
                print("len(fluxes_list) = ", len(fluxes_list))
                
                print(len(metrics_list))
                for metric in metrics_list:
                    if "metric" != "feasibility":
                        fluxes_list.append(np.nan)
                    else:
                        fluxes_list.append(int(0))

                print("\033[91m"+"Model iteration is infeasible!\033[00m")
                
                const_list.append(int(0)) #to add 0 to feasibility
                const_list.append(np.nan) #to add 0 to nCO2


        else:
        # runs when model optimisation produced feasible result
        # saves fluxes of each reactions, and value of metric to add to dataframe


            print("Model iteration feasible. Objective flux:", round(temp_model.reactions.get_by_id(objective_reaction).flux,7))
            for rxn in temp_model.reactions:
                flux = rxn.flux
                if "linker_12_to_00" in rxn.id:
                    flux = (-1)*flux #to make night to day accumulation negative for plotting later

                fluxes_list.append(flux)

            #noct. CO2 coeff.
            CO2_coeff = calculate_noct_CO2_refixation_coefficient(temp_model, verbose=True)
            fluxes_list.append(CO2_coeff)
            print("nCO2RC = ", CO2_coeff)

            #CCE
            CCE = calculate_CCE(temp_model)
            fluxes_list.append(CCE)

            #calculating sum of all fluxes
            sum_of_fluxes = calculate_sum_of_all_fluxes(temp_sol)
            fluxes_list.append(sum_of_fluxes)

            #calculating photon use
            fluxes_list.append(temp_model.reactions.Photon_tx_00.flux/temp_model.reactions.Photon_tx_00.upper_bound)

            #objective, photon, CO2 upper bound
            fluxes_list.append(temp_model.reactions.get_by_id(objective_reaction).upper_bound)
            fluxes_list.append(temp_model.reactions.Photon_tx_00.upper_bound)
            fluxes_list.append(temp_model.reactions.CO2_tx_00.upper_bound)

            #total amino acid accumulation
            amino_acids = ["GLN_v","VAL_v","PHE_v","TYR_v","L_ALPHA_ALANINE_v","SER_v",
                       "TRP_v","GLT_v","ILE_v","L_ASPARTATE_v","CYS_v","MET_v","bHIS_v",
                       "ARG_v","THR_v","LEU_v","PRO_v","LYS_v","ASN_v","GLY_v"]
            aa_sum_day = 0
            aa_sum_night = 0
            for item in amino_acids:
                for rxn in temp_model.reactions:
                    if item in rxn.id and "linker_00_to_12" in rxn.id:
                        aa_sum_day += rxn.flux
                    elif item in rxn.id and "linker_12_to_00" in rxn.id:
                        aa_sum_night += rxn.flux
            fluxes_list.append(aa_sum_day)
            fluxes_list.append((-1)*aa_sum_night)

            #save whether model was feasible or not
            fluxes_list.append(int(1)) #1 means feasible (which I decided to use rather than a string)
            const_list.append(int(1))
            const_list.append(CO2_coeff) #add nCO2 refixation coefficient to the constraint list

            
            # save current solution to solution_dict
            solution_dict[md]= temp_sol


        #name column in dataframe after current model and insert fluxes/metric values
        column_name = md
        df.insert(iteration+1, column_name, fluxes_list)
        print(const_list)
        df_constraints.insert(iteration+1, column_name, const_list)

        print(const_list)
    df.set_index("reactions", inplace= True)
    df_constraints.set_index("Constraint", inplace= True)
    
    return df, df_constraints


In [58]:
def set_NGAM_value(model, lower_bound=None, upper_bound=None, light_tag="_00"):
    """
    sets the bounds of ATPase_tx in the light phase
    """
    #assert that specified lower bound has smaller value than upper bound
    if upper_bound != None:
        assert lower_bound < upper_bound, "The 1st value (lower bound) must be smaller than the 2nd value (upper_bound).\n Fix this and try again!"

    #if only one value is specified, set values of both lower bound and upper bound to be the same    
    if upper_bound == None:
        upper_bound = lower_bound
    

    model.reactions.get_by_id("ATPase_tx"+light_tag).bounds = (lower_bound, upper_bound)    

    return lower_bound, upper_bound



def set_CO2_tx_day(model, lower_bound=None, upper_bound=None, light_tag="_00"):
    """
    sets the bounds of CO2_tx in the light phase
    """
    #assert that specified lower bound has smaller value than upper bound
    if upper_bound != None:
        assert lower_bound < upper_bound, "The 1st value (lower bound) must be smaller than the 2nd value (upper_bound).\n Fix this and try again!"

    #if only one value is specified, set values of both lower bound and upper bound to be the same    
    if upper_bound == None:
        upper_bound = lower_bound
    

    model.reactions.get_by_id("CO2_tx"+light_tag).bounds = (lower_bound, upper_bound)    

    return lower_bound, upper_bound

def set_nitrate_uptake_day_night_ratio(model, ratio):
    model.remove_reactions(["Nitrate_tx_light", "Nitrate_tx_dark"])
    
    
    nitrate_light_rxn = Reaction("Nitrate_tx_light")
    nitrate_light_rxn.add_metabolites({model.metabolites.Nitrate_light: 1,
                                      model.metabolites.Nitrate_total: -1})
    nitrate_light_rxn.upper_bound = 1000
    nitrate_light_rxn.lower_bound = -1000
    model.add_reactions([nitrate_light_rxn])

    
    nitrate_dark_rxn = Reaction("Nitrate_tx_dark")
    nitrate_dark_rxn.add_metabolites({model.metabolites.Nitrate_dark: 1,
                                      model.metabolites.Nitrate_total: -1})
    nitrate_dark_rxn.upper_bound = 1000
    nitrate_dark_rxn.lower_bound = -1000
    model.add_reactions([nitrate_dark_rxn])

    nitrate_light_dark_const = model.problem.Constraint(
                            (model.reactions.Nitrate_tx_light.flux_expression) - 
                            ratio*(model.reactions.Nitrate_tx_dark.flux_expression), 
                            lb = 0,
                            ub= 0)
    model.add_cons_vars(nitrate_light_dark_const)
    
    return ratio, "test"


def set_Rubisco_C_to_O_ratio(model, ratio):
    """           # this constraint sets the ratio between Rubisco carboxylation and oxygenation
                    
                    for i in range(len(phases_list)):
                        print(i, phases_list[i])
                        
                        # only sets a constraint if there is light in that phase
                        if model.reactions.get_by_id("Photon_tx_"+phases_list[i][1:]).upper_bound > 0 :
                        
                            rubisco_c_to_o_const = model.problem.Constraint(
                            ratio[1]*(model.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_"+phases_list[i][1:]).flux_expression) -
                            ratio[0]* model.reactions.get_by_id("RXN_961_p_"+phases_list[i][1:]).flux_expression,
                            lb = 0,
                            ub = 0,
                            name="rubisco_c_to_o_const"+phases_list[i][1:])
                            model.add_cons_vars(rubisco_c_to_o_const)
                            print("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_"+phases_list[i][1:])
                            print("RXN_961_p_"+phases_list[i][1:])
                    print("Rubisco CO constraint:", ratio)
    """

    
    model.remove_cons_vars(model.constraints.rubisco_c_to_o_const00)
    
    rubisco_C_O = model.problem.Constraint(
                            (model.reactions.RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_00.flux_expression) - 
                            ratio*(model.reactions.RXN_961_p_00.flux_expression), 
                            lb = 0,
                            ub= 0)
    model.add_cons_vars(rubisco_C_O)
    return ratio, "test"

def set_NGAM_ATP_NADPH_ratio(model, ratio):
    """    #             this constraint sets ration between ATPase and the NADPH oxidases in each time phase
                    print("ATP to NADPH maintenance ratio:", all_or_ratio)
                    for i in range(len(phases_list)):
                        ATPase_NADPHox_const = model.problem.Constraint(
                        ratio[1]*(model.reactions.get_by_id("ATPase_tx_"+phases_list[i][1:]).flux_expression) - 
                            ratio[0]*(model.reactions.get_by_id("NADPHoxc_tx_"+phases_list[i][1:]).flux_expression +
                              model.reactions.get_by_id("NADPHoxm_tx_"+phases_list[i][1:]).flux_expression+
                              model.reactions.get_by_id("NADPHoxp_tx_"+phases_list[i][1:]).flux_expression ), 
                        lb = 0,
                        ub= 0,
                        name="ATPase_to_NADPHox_const"+phases_list[i][1:])
                        model.add_cons_vars(ATPase_NADPHox_const)
    """


    model.remove_cons_vars(model.constraints.ATPase_to_NADPHox_const00)
    
    ATPase_NADPHox_const00 = model.problem.Constraint(
                        (model.reactions.get_by_id("ATPase_tx_00").flux_expression) - 
                            ratio*(model.reactions.get_by_id("NADPHoxc_tx_00").flux_expression +
                              model.reactions.get_by_id("NADPHoxm_tx_00").flux_expression+
                              model.reactions.get_by_id("NADPHoxp_tx_00").flux_expression ), 
                        lb = 0,
                        ub= 0,
                        name="ATPase_to_NADPHox_const00")
    model.add_cons_vars(ATPase_NADPHox_const00)
        
    
    model.remove_cons_vars(model.constraints.ATPase_to_NADPHox_const12)
    ATPase_NADPHox_const12 = model.problem.Constraint(
                    (model.reactions.get_by_id("ATPase_tx_12").flux_expression) - 
                        ratio*(model.reactions.get_by_id("NADPHoxc_tx_12").flux_expression +
                          model.reactions.get_by_id("NADPHoxm_tx_12").flux_expression+
                          model.reactions.get_by_id("NADPHoxp_tx_12").flux_expression ), 
                    lb = 0,
                    ub= 0,
                    name="ATPase_to_NADPHox_const12")
    model.add_cons_vars(ATPase_NADPHox_const12)
                          
    return ratio, "test"


def set_NGAM_day_night_ratio(model, ratio):
    model.remove_cons_vars(model.constraints.NGAM_day_night_const)
    ATPase_light_dark = model.problem.Constraint(
                            (model.reactions.get_by_id("ATPase_tx_light").flux_expression) - 
                            ratio*(model.reactions.get_by_id("ATPase_tx_dark").flux_expression), 
                            lb = 0,
                            ub= 0,
                            name="NGAM_day_night_const")
    model.add_cons_vars(ATPase_light_dark)
    return ratio, "test"

def set_biomass_day_night_ratio(model, ratio, biomass_output_string = "Marchantia_Naomi_minus_starch_tx"):
    """
    sets the ratio between the daytime biomass reaction and night-time biomass reaction
    """
    
    try:
        model.remove_cons_vars(model.constraints.biomass_day_night_const)
    except:
        pass
                                  
    biomass_light_dark_const = model.problem.Constraint(
                            (model.reactions.get_by_id(biomass_output_string + "_light").flux_expression) - 
                            ratio*(model.reactions.get_by_id(biomass_output_string + "_dark").flux_expression), 
                            lb = 0,
                            ub= 0,
                            name="biomass_day_night_const")
    model.add_cons_vars(biomass_light_dark_const)
                          
    return ratio, "test"
                          
                          
def set_objective_flux_ub(model, obj_flux):
    """
    model.reactions.get_by_id(objective_reaction).bounds = (obj_flux, obj_flux)
    return obj_flux"""
    pass

def set_estimate_Output(model, buffer):
    global estimate_Output 
    # estimate_Output = not estimate_Output
    estimate_Output = True
    if estimate_Output:
        return 1, "test"
    else:
        return 0, "test"

In [23]:
def create_x_model_objects(reference_model, nr_models):
    """
    function to create x number of models as model objects with distinct names (00-xx)
    """
    print("Creating", nr_models, "models...")
    
    assert type(nr_models) == int
    digits = len(str(nr_models))
    
    model_dict = {}
    model = reference_model
    
    for i in range(nr_models):
        exec(f'model_{str(i).zfill(digits)} = model.copy()')
        model_dict["model_"+str(i).zfill(digits)] = eval(f'model_{str(i).zfill(digits)}')
    
    print(nr_models, "models created.")

    return model_dict

In [ ]:
def find_necess_nr_of_models(mode, constraints_list, iterations):
    """
    function to determine the necessary number of models to cover all the desired model parameter combinations/variations
    """
    
    # mode = "cumulative", "combination" or "variation"
    assert mode in ["cumulative", "combination", "variation"], 'mode parameter must be "cumulative", "combination" or "variation"'
    
    if mode == "variation":
        nr_of_models = iterations +1
    elif mode == "combination":
        nr_of_models = len(constraints_list)**2
    elif mode == "cumulative":
        nr_of_models = len(constraints_list)+1
        
    return nr_of_models

In [ ]:
def perform_sensitivity_analysis(reference_model, param_ranges_list, optimisation_method, iterations):
    """
    function to perform sensitivity analysis, by running x iterations of the same model with certain parameters varied in specified ranges
    """
    
    # generate sobol sequences
    param_ranges_sobol_list = generate_sobol_sequences(param_ranges_list, iterations)

    #create as many model objects as needed
    model_dict = create_x_model_objects(reference_model, iterations)

def generate_sobol_sequences(problem, number_of_sobol_numbers):
    """
    Function to generate quasirandom sequences using sobol for uniform distribution
    - input:
        dict containing parameter names, and set of min and max parameter ranges
    - output:
        dict contatining parameter names and list of sobol numbers for each specified range
    """
    from scipy.stats import qmc
    
    parameter_sobol_dict = {}
    problem["sobol_sequences"] = []
    
    for x in range(len(problem["parameter_names"])):
        qrng = qmc.Sobol(d=number_of_sobol_numbers, scramble=True)
        sample_qmc = qrng.random(n=1)
        
        sample_continued = qmc.scale(sample_qmc, problem["bounds"][x][0], problem["bounds"][x][1])
        seq_list = sample_continued.tolist()[0]
        
    
        problem["sobol_sequences"].append(seq_list)
    return problem

In [43]:
def generate_sobol_sequences(problem, iterations):
    """
    Function to generate quasirandom sequences using sobol for uniform distribution
    - input:
        dict containing parameter names, and set of min and max parameter ranges
    - output:
        dict contatining parameter names and list of sobol numbers for each specified range
    """
    from scipy.stats import qmc
    
    dimensions = len(problem["parameter_functions"])
    
    parameter_sobol_dict = {}
    problem["sobol_sequences"] = []
    
    counter = 0
    while 2**counter < iterations:
        counter+=1
    else:
        nr_sobol_seq = 2**counter
    
    qrng = qmc.Sobol(d=dimensions)
    sample_qmc = qrng.random(n=nr_sobol_seq)
    print(sample_qmc)
    
    # for sequence in sample_qmc:        
        # sample_continued = qmc.scale(sample_qmc, problem["bounds"][0][0], problem["bounds"][0][1])
    
    sample_cont = qmc.scale(sample_qmc, problem['l_bounds'], problem['u_bounds'])
    problem["sobol_sequences"] = sample_cont.tolist()
    
    return problem

In [25]:
def generate_random_sequences(problem, number_of_random_numbers):
    pass
    
    """
    Function to generate random sequences
    - input:
        dict containing parameter names, and set of min and max parameter ranges
    - output:
        dict contatining parameter names and list of random numbers for each specified range
    """
    problem["random_sequences"] = []
    
    for x in range(len(problem["parameter_names"])):
        random_list =[]
        
        for i in range(number_of_random_numbers):
            random_nr = random.uniform(problem["bounds"][x][0], problem["bounds"][x][1])
            random_list.append(random_nr)
    
        problem["random_sequences"].append(random_list)
    return problem

In [13]:
def apply_constraints_to_model(model, constraints_dict):
    pass

In [ ]:
def calculate_sum_of_all_fluxes(solution):
    sum_of_fluxes = 0
    for item in solution:
        sum_of_fluxes += abs(item)

    return sum_of_fluxes

In [ ]:
def count_df_columns_condition(df, index):
    
    df_index= df.loc[index]
    nr_df_col_true = len(df_index[df_index==1])
    
    return nr_df_col_true

In [12]:
### calculating how much of nocturnally produced CO2 is refixed and how much is released ###

def calculate_noct_CO2_refixation_coefficient(model, verbose=False, threshold = 0.00001, show_noflux = False):

    CO2_c2_produced = 0
    CO2_c2_consumed = 0
    
    organelles = {"Cytosol":"_c","Plastid": "_p","Mitochondrion": "_m", "Peroxisome":"_x"}

    CO2_rxns = []

    for rxn in model.reactions:
        #find all nocturnal reactions that involve CO2, except the import from external into cytosol
        for met in rxn.metabolites:
            current_met = met.id
            if ("CARBON_DIOXIDE_" in met.id or "HCO3_" in met.id) and met.id[-3:]=="_12" and "CO2_ec" not in rxn.id and rxn.id not in CO2_rxns:
                CO2_rxns.append(rxn.id)
            # if re.search('CARBON_DIOXIDE_._12', met.id) and (rxn.id != "CO2_tx_12" and rxn.id != "CO2_ec_12"):
            if re.search('CARBON_DIOXIDE_._12', met.id) and (rxn.id != "CO2_tx_12" and rxn.id != "CO2_ec_12" and rxn.id != "CO2_mc_12" and rxn.id != "CO2_pc_12" and rxn.id != "CO2_xc_12"):
                CO2_flux = rxn.metabolites.get(model.metabolites.get_by_id(current_met)) * rxn.flux
                if CO2_flux > 0:
                    CO2_c2_produced += CO2_flux
                    # print("produced:", rxn.id, CO2_flux)

                elif CO2_flux < 0:
                    CO2_c2_consumed += CO2_flux
                    # print("consumed:", rxn.id, CO2_flux)

    for item in organelles:
        print("-----", item, "-----")
        for rxn in CO2_rxns:
            for met in model.reactions.get_by_id(rxn).metabolites:
                if "CARBON_DIOXIDE"+organelles[item] in met.id: 
                    CO2_flux = model.reactions.get_by_id(rxn).flux * model.reactions.get_by_id(rxn).get_coefficient(met)
                    if CO2_flux > 0 and abs(CO2_flux) > threshold:
                        if verbose:
                            print("\033[96m"+"producing CO2:\t"+ rxn+ "\t"+str(round(model.reactions.get_by_id(rxn).flux, 6))+"\033[00m")
                    elif CO2_flux < 0 and abs(CO2_flux) > threshold:
                        if verbose:
                            print("\033[91m"+"consuming CO2:\t"+ rxn+ "\t"+str(round(model.reactions.get_by_id(rxn).flux, 6))+"\033[00m")
                    else:
                        if verbose and show_noflux:
                            print("\033[97m"+"not consuming/producing CO2:\t"+ rxn + "\t"+str(model.reactions.get_by_id(rxn).flux) + "\033[00m")
                elif "HCO3"+organelles[item] in met.id: 
                    CO2_flux = model.reactions.get_by_id(rxn).flux * model.reactions.get_by_id(rxn).get_coefficient(met)
                    if CO2_flux > 0 and abs(CO2_flux) > threshold:
                        if verbose:
                            print("\033[96m"+"producing HCO3-:\t"+ rxn+ "\t"+str(round(model.reactions.get_by_id(rxn).flux, 6))+"\033[00m")
                    elif CO2_flux < 0 and abs(CO2_flux) > threshold:
                        if verbose:
                            print("\033[91m"+"consuming HCO3-:\t"+ rxn+ "\t"+str(round(model.reactions.get_by_id(rxn).flux, 6))+"\033[00m")
                    else:
                        if verbose and show_noflux:
                            print("\033[97m"+"not consuming/producing HCO3-:\t"+ rxn + "\t"+str(model.reactions.get_by_id(rxn).flux) + "\033[00m")
                            
                

    CO2_exchange = model.reactions.CO2_ec_12.flux


    if verbose:
        print("\nCO2 produced:", round(CO2_c2_produced, 5))
        print("CO2 consumed:", round(CO2_c2_consumed,5))
        print("CO2 exchange:", round(CO2_exchange, 5))

    try:
        # CO2_metric = (abs(CO2_c2_consumed)-abs(CO2_exchange))/abs(CO2_c2_produced)
        CO2_metric = abs(CO2_c2_consumed)/abs(CO2_c2_produced)

    except:
        CO2_metric = 0
    if verbose:
        print("\nnocturnal CO2 refixation coeffient:", round(CO2_metric,5))
    return CO2_metric

In [8]:
###

def determine_obj_reached(df, df_con, objective_reaction, objective_goal):
    expt_bool_list = []
    expt_list = []

    count = 0
    for item in df.loc[objective_reaction]:
        proportion_expt_obj = item/objective_goal
        print(count,":",proportion_expt_obj)
        if proportion_expt_obj >= 1:
            expt_obj_bool = 1
        else:
            expt_obj_bool = 0
        expt_bool_list.append(expt_obj_bool)
        expt_list.append(proportion_expt_obj)
        count += 1

    df_con.loc["expt. gr reached"] = expt_bool_list
    df_con.loc["proportion expt. gr reached"] = expt_list

    df_plot = df_con.transpose()

    df_plot = df_plot.drop(df_plot.iloc[0]._name) #to remove first iteration in which no parameters were varied
    
    return df_plot

In [ ]:
###

def parallel_coord_plot(df, dont_pl):

    import plotly.graph_objects as go
    log_plot = False
    colour_ref = "expt. gr reached"
    colour_min = 'lightgreen'

    df_plot.columns = ['NGAM_costs', 'CO2_uptake', 'Nit_d-n', 'Rubisco_C-O', 'ATP-NADPH', 'NGAM_d-n', 'biomass_d-n', 'feasibility', 'expt. gr reached']
    dont_plot = ["estimateOutput", "feasibility", "expt. gr reached", 'estimate_output']

    if log_plot == True:

        #take log of each value
        # df = df_plot
        df_log = df_plot.copy()

        for col in df_plot:
            if col not in ["feasibility", "expt. gr reached"]:
                count = 0
                for item in df_plot[col]:
                    try:
                        df_log[col][count] = math.log(item)
                    except:
                        pass
                    count+=1

        df = df_log
    else:
        df = df_plot

    dimensions = []
    for col in df.columns:
        if col not in dont_plot:
            dimensions.append(dict(range= [round(df[col].min(),1), round(df[col].max(),1)], values = df[col], label = col))

    fig = go.Figure(data=
        go.Parcoords(
            line = dict(color = df[colour_ref]
                        ,colorscale = [[0.0,colour_min],[1.0,'black']],
                        showscale = True
                       ),
    #         dimensions = list([
    #             dict(range = [0,8],
    #                 constraintrange = [4,8],
    #                 label = 'CO2_tx_day', values = df['randomise_CO2_tx_day']),
    #             dict(range = [0,df2['randomise_NGAM_value'].max()],
    #                 label = 'NGAM', values = df['randomise_NGAM_value']),
    #             dict(range = [0,df2['randomise_nitrate_linker_rate'].max()],
    #                 label = 'Nitrate linker', values = df['randomise_nitrate_linker_rate']),
    #             dict(range = [0,8],
    #                 label = 'Nitrate ratio', values = df['randomise_nitrate_uptake_day_night_ratio'])
    #         ])
            dimensions = dimensions
        )
    )

    fig.update_layout(
        plot_bgcolor = 'white',
        paper_bgcolor = 'white'
    )

    fig.show()

In [22]:
def calculate_CCE(model):
    CO2_day = model.reactions.CO2_tx_00.flux
    CO2_night = model.reactions.CO2_tx_12.flux
    
    if CO2_night <= 0:
        try:
            CCE = 1 - (abs(CO2_night)/CO2_day)
        except ZeroDivisionError:
            CCE = 0
    else:
        CCE = 1
        
    return CCE

In [23]:
def check_rxns_with_met_x(model, met, round_fig = 5):
    for rxn in model.metabolites.get_by_id(met).reactions:
        print(rxn.id, round(rxn.flux,round_fig))


In [24]:
def parameter_scan(model, objective, process, start, stop, stepsize):
    temp_model = model.copy()
    
    rxn_list = []
    for rxn in model.reactions:
        rxn_list.append(rxn.id)
    model_rxns = {"reactions": rxn_list}
    
    df = pd.DataFrame(model_rxns)
    print("Scanning process:", process,"\n")
    for count, value in enumerate(np.arange(start,stop,stepsize)):
        print("Iteration:", count,"    ", "Scan value:", value)
        try: #order of which bound to set first depends on previously specified bounds, e.g. lb always needs to be lower than current ub
            temp_model.reactions.get_by_id(process).lower_bound = 0
            temp_model.reactions.get_by_id(process).upper_bound = value
        except:
            temp_model.reactions.get_by_id(process).upper_bound = value
            temp_model.reactions.get_by_id(process).lower_bound = 0
        
        #try optimising model
        #if infeasible, save every flux as 0
        fluxes_list =[]
        try:
            solution_temp, temp_model = optimise_model(temp_model, objective)
            #adding a new column with the new fluxes to the dataframe
            for rxn in temp_model.reactions:
                flux = rxn.flux
                if "linker_12_to_00" in rxn.id:
                    flux = (-1)*flux #to make night to day accumulation negative for plotting later

                fluxes_list.append(flux)
        except:
            for rxn in temp_model.reactions:
                fluxes_list.append(0)
            print("\033[91m"+"Model iteration "+ str(count)+ " is infeasible!"+"\033[00m")
        
        df.insert(count+1,"Result_"+str(count+1),fluxes_list)
        print("\n")
        
        
        
    df.set_index("reactions", inplace= True)
    return df

In [25]:
def parameter_scan_CO2(model, objective, process, start, stop, stepsize, pFBA, CO2_refixation_allowed=True, verbose=False, iterations=10):
    solution_dict = {}
    
    # make list of all reactions to save in the dataframe
    rxn_list = []
    for rxn in model.reactions:
        rxn_list.append(rxn.id)
    # add other metrics to save in dataframe
    metrics_list = ["noct. CO2 coeff.", "CCE", "sum of fluxes", "photon use", "growth rate ub", "photon use ub", "daytime CO2 uptake ub", "scan process ub", "total aa day", "total aa night"]
    for metric in metrics_list:
        rxn_list.append(metric)
        
    
    # create dataframe (rows: reactions and other metrics; columns: flux/value for each model iteration with specified scan value)
    model_rxns = {"reactions": rxn_list}
    df = pd.DataFrame(model_rxns)
    
    # start scan by changing upper bound of specific reaction to current scan value
    print("Scanning process:", process,"\n")
    
    for count, scan_value in enumerate(np.arange(start,stop,stepsize)):
        temp_model = model.copy()
        
        print("------- Scan iteration:", count+1,"    ", "Scan value:", scan_value, "("+process+") --------")
        
        
        # try to optimise the model
        fluxes_list =[]
        try:
            if CO2_refixation_allowed:
                try: #order of which bound to set first depends on previously specified bounds, e.g. lb always needs to be lower than current ub
                    temp_model.reactions.get_by_id(process).lower_bound = 0
                    temp_model.reactions.get_by_id(process).upper_bound = scan_value
                except:
                    temp_model.reactions.get_by_id(process).upper_bound = scan_value
                    temp_model.reactions.get_by_id(process).lower_bound = 0
                    
                solution_temp, opt_model = optimise_model(temp_model, objective, pFBA)
            else:
                solution_temp, opt_model = estimateOutputFromNetCO2_Corinna_dev(temp_model,netCO2uptake=scan_value,Output_ID=objective,
                                      Vc_ID="RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_00",
                                      CO2in_ID="CO2_tx_00",verbose=verbose, iterations=iterations, pFBA=pFBA)
        except:
            # if infeasible, save every flux and metric as NaN to add to dataframe
            for rxn in temp_model.reactions:
                fluxes_list.append(np.nan)
            for metric in metrics_list:
                fluxes_list.append(np.nan)
            print("\033[91m"+"Model iteration is infeasible!\033[00m")
            
            
        else:
        # runs when model optimisation produced feasible result
        # saves fluxes of each reactions, and value of metric to add to dataframe
            print("Model iteration feasible. Objective flux:", round(opt_model.reactions.get_by_id(objective).flux,7))
            for rxn in opt_model.reactions:
                flux = rxn.flux
                if "linker_12_to_00" in rxn.id:
                    flux = (-1)*flux #to make night to day accumulation negative for plotting later

                fluxes_list.append(flux)
            
            #noct. CO2 coeff.
            CO2_coeff = calculate_noct_CO2_refixation_coefficient(opt_model, verbose=verbose)
            fluxes_list.append(CO2_coeff)
            
            #CCE
            CCE = calculate_CCE(opt_model)
            fluxes_list.append(CCE)
            
            #calculating sum of all fluxes
            sum_of_fluxes = 0
            for item in solution_temp:
                sum_of_fluxes += abs(item)
            fluxes_list.append(sum_of_fluxes)
            
            #calculating photon use
            fluxes_list.append(opt_model.reactions.Photon_tx_00.flux/opt_model.reactions.Photon_tx_00.upper_bound)
            
            #objective, photon, CO2 upper bound
            fluxes_list.append(opt_model.reactions.get_by_id(objective).upper_bound)
            fluxes_list.append(opt_model.reactions.Photon_tx_00.upper_bound)
            fluxes_list.append(opt_model.reactions.CO2_tx_00.upper_bound)
            
            #scan reaction upper bound
            fluxes_list.append(opt_model.reactions.get_by_id(process).upper_bound)
            
            #total amino acid accumulation
            amino_acids = ["GLN_v","VAL_v","PHE_v","TYR_v","L_ALPHA_ALANINE_v","SER_v",
                       "TRP_v","GLT_v","ILE_v","L_ASPARTATE_v","CYS_v","MET_v","bHIS_v",
                       "ARG_v","THR_v","LEU_v","PRO_v","LYS_v","ASN_v","GLY_v"]
            aa_sum_day = 0
            aa_sum_night = 0
            for item in amino_acids:
                for rxn in opt_model.reactions:
                    if item in rxn.id and "linker_00_to_12" in rxn.id:
                        aa_sum_day += rxn.flux
                    elif item in rxn.id and "linker_12_to_00" in rxn.id:
                        aa_sum_night += rxn.flux
            fluxes_list.append(aa_sum_day)
            fluxes_list.append((-1)*aa_sum_night)
                        

            solution_dict[count]= solution_temp
        
        #name model iteration after current scan value
        column_name = str(round(scan_value, 3))
        df.insert(count+1, column_name, fluxes_list)

    
    df.set_index("reactions", inplace= True)
    
    print("\n")
    return df, solution_dict

In [26]:
def pearsons_correlation(cobra_model, solutiondict, PCC_thresh, p_thresh):
    keyList = list(solutiondict.keys())
    rxnvalues=dict()

#     for rxn in cobra_model.reactions:
#         values=list()
#         for i in keyList:
#             values.append(solutiondict[i][rxn.id])
#         rxnvalues[rxn.id]=values

    for rxn in solution_dict1[0].fluxes.keys():
        values=list()
        for i in keyList:
            values.append(solutiondict[i][rxn])
        rxnvalues[rxn]=values


    from scipy import stats

    PCCdict=dict()
    i=0;
    for rxn1 in solution_dict1[0].fluxes.keys():
        print("Processing "+str((i+1))+" of "+str(len(solution_dict1[0].fluxes.keys())))
        i+=1
        PCC=dict()
        for rxn2 in solution_dict1[0].fluxes.keys():
            PCC[rxn2] = stats.pearsonr(rxnvalues[rxn1],rxnvalues[rxn2])
        PCCdict[rxn1]=PCC

    usedSet=set()
    clustered=set()

    for rxn1 in PCCdict.keys():
        if rxn1 not in usedSet:
            usedSet.add(rxn1)
            tempset=set()
            tempset.add(rxn1)
            for rxn2 in PCCdict.get(rxn1):
                if((PCCdict.get(rxn1).get(rxn2)[0] > PCC_thresh or PCCdict.get(rxn1).get(rxn2)[0] < PCC_thresh*(-1)) and PCCdict.get(rxn1).get(rxn2)[1] < p_thresh): #to cluster also negative correlations
#                 if((PCCdict.get(rxn1).get(rxn2)[0] > PCC_thresh) and PCCdict.get(rxn1).get(rxn2)[1] < p_thresh):

                    usedSet.add(rxn2)
                    tempset.add(rxn2)
            clustered.add(frozenset(tempset))

    grouping = dict()
    i=0
    for s in clustered:
        if(len(s)>1):
            grouping[i]=s
            i+=1

    return rxnvalues, clustered, grouping, PCCdict

In [38]:
def plot_accum(dataframe, threshold, growth_rate, CO2_day_rate, lim2_x=0, lim2_y=1.4, lim3_x=-1.5, lim3_y=7.2, lim5_x=-0.05, lim5_y=0.05, time_phase="_12", CO2_organelle = "_c"):
    model = phased_model
    results = dataframe.columns.tolist()
    threshold = 0.0001

    ### Extracting fluxes from linker reactions ##################################################################################
    y_values = {}
    y_values_pos = {}
    y_values_neg = {}

    for index, item in dataframe.iterrows():
        x_list = []
        x_pos = []
        x_neg = []
        if "linker" in index:
            sum = 0
            for x in item:
                if not np.isnan(x):
                    sum += x
        for x in item:
    #         x_list.append(round(abs(x),5))
            x_list.append(round(x,5))
            if x >= 0:
                x_pos.append(round(x,5))
                x_neg.append(0)
            elif x <= 0:
                x_neg.append(round(x,5))
                x_pos.append(0)

        if "linker" in index and sum != 0 and (sum>threshold or sum<(threshold*(-1))):  
            y_values[index] = x_list
            y_values_pos[index] = x_pos
            y_values_neg[index] = x_neg

    ### Extracting reactions that involve nocturnal, cytosolic CO2 ##################################################################################

    CO2_12_fluxes = {}
    total_CO2_produced = [0] * len(dataframe.columns)
    total_CO2_consumed = [0] * len(dataframe.columns)

    CO2_metabolite = "CARBON_DIOXIDE"

    # iterate over each row in the dataframe
    for index, item in dataframe.iterrows():
        CO2_x_list = []
        try:
            # iterate over reactions in the model to find any that contain the CO2 metabolite
            for met in model.reactions.get_by_id(index).metabolites:
                if met.id[:-5] == CO2_metabolite and met.id[-5:-3] in CO2_organelle and met.id[-3:] == time_phase and "_pc_" not in index and "_xc_" not in index and "_mc_" not in index and "_ec_" not in index:
                    CO2_met = met.id
                    sum = 0
                    
                    # for each CO2 metabolite containing reaction, get the flux for each model iteration and sum them up separately for production and consumption
                    for count, x in enumerate(item):
                        if not np.isnan(x):
                            y = x * model.reactions.get_by_id(index).get_coefficient(CO2_met)
                            sum += y
                            CO2_x_list.append(y)   
                            if y > 0:
                                total_CO2_produced[count] += y
                            elif y < 0:
                                total_CO2_consumed[count] += y
                        else:
                            CO2_x_list.append(np.nan)   
                    CO2_12_fluxes[index] = CO2_x_list

            CO2_12_fluxes["total CO2 produced"] = total_CO2_produced
            CO2_12_fluxes["total CO2 consumed"] = total_CO2_consumed
        except:
            #exception needed in cases where metric was added to dataframe (eg noct CO2 refixation coefficient), as they don't have .metabolites attribute
            continue
            



    ######################## Plotting ###########################################################################################################################           

    fig, (ax2, ax3, ax5, ax4) = plt.subplots(4, sharex = True)

    ax2.set_title("Objective")
    ax2.plot(results, dataframe.loc[objective_reaction], label="growth rate", linewidth=2.2)

    ax2.plot(results, dataframe.loc["sum of fluxes"]/1000, label="sum of fluxes/1000")
    ax2.axhspan(growth_rate[0], growth_rate[1], color='gray', alpha=0.3, lw=0)
    ax2.set_ylim((0,1.1))
    ax2.plot(results, dataframe.loc["photon use"], label="photon use")
    ax2.plot(results, dataframe.loc["growth rate ub"], "--", label="growth rate ub", linewidth=1.2, color="gray")

    ax2.legend(loc="center left", bbox_to_anchor=(1, 0.5))


    ax3.set_title("CO2 Exchange")
    ax3.plot(results, dataframe.loc["CO2_tx_00"], label="CO2 day", linewidth=2)
    ax3.plot(results, dataframe.loc["daytime CO2 uptake ub"], "--", label="CO2 day ub", linewidth=1.2, color="gray")

    ax3.plot(results, dataframe.loc["CO2_tx_12"], label="CO2 night")
    ax3.plot(results, dataframe.loc["noct. CO2 coeff."], label="CO2 refix. coeff.", linewidth=2.2)
    ax3.plot(results, dataframe.loc["CCE"],"-.", label="CCE")
    ax3.plot(results, dataframe.loc["RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_00"], label="Rubisco")
    ax3.plot(results, dataframe.loc["ISOCITDEH_RXN_c_12"], label="Isocit deh_c night")
    ax3.plot(results, dataframe.loc["RXN0_5224_c_12"], label="carb anhyd_c night")
    ax3.plot(results, dataframe.loc["PEPCARBOX_RXN_c_12"],"--", label="PEPC_c night")
    ax3.plot(results, dataframe.loc["PEPCARBOX_RXN_c_00"],"--", label="PEPC_c day")
    # ax3.plot(results, dfx.loc["PEPCARBOXYKIN_RXN_c_12"], label="PEPCkinase_c night")

    ax3.axhspan(CO2_day_rate[0], CO2_day_rate[1], color='gray', alpha=0.3, lw=0)

    ax3.legend(loc="center left", bbox_to_anchor=(1, 0.5))
    

    #display nocturnal CO2 involving reaction fluxes
    ax4.set_title("nocturnal CO2 production/consumption")
    
    LINE_STYLES = ['solid', 'dashed', 'dashdot', 'dotted']
    NUM_COLORS = len(CO2_12_fluxes)
    cm = plt.get_cmap('tab20')
    ax4.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)], linestyle= [LINE_STYLES[i%len(LINE_STYLES)] for i in range(NUM_COLORS)])

    for rxn in CO2_12_fluxes:
        width = 2
        sum_CO2 = 0
        for item in CO2_12_fluxes[rxn]:
            sum_CO2 += item
        if sum_CO2 == 0:
            width = 0.05
        if sum_CO2 != 0:
            if "total" in rxn:
                ax4.plot(results, CO2_12_fluxes[rxn],"-.", label=rxn, linewidth=1)
            else:
                ax4.plot(results, CO2_12_fluxes[rxn], label=rxn, linewidth=width)
    ax4.legend(loc="center left", bbox_to_anchor=(1, 0.5))


    # display accumulation related fluxes
    ax5.set_title("Accumulation")

    LINE_STYLES = ['solid', 'dashed', 'dashdot', 'dotted']
    NUM_STYLES = len(LINE_STYLES)
    NUM_COLORS = len(y_values)
    ax5.set_title("Accumulation")
    cm = plt.get_cmap('gist_rainbow')
    ax5.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)], linestyle= [LINE_STYLES[i%len(LINE_STYLES)] for i in range(NUM_COLORS)])
    ax5.plot(results, dataframe.loc["total aa day"],"--", label="total aa day", linewidth=1.2, color="gray")
    ax5.plot(results, dataframe.loc["total aa night"],"--", label="total aa night", linewidth=1.2, color="gray")

    


    for values in y_values:
        ax5.plot(results, y_values[values], label=values[:-16], linewidth=2)
    ax5.legend(loc="center left", bbox_to_anchor=(1, 0.5))


    ax3.set_xticks(range(len(dataframe.columns)))
    ax3.set_xticklabels(dataframe.columns, rotation=90)
    ax2.set_xticks(range(len(dataframe.columns)))
    ax2.set_xticklabels(dataframe.columns, rotation=90, alpha=0)

    plt.xticks(range(len(dataframe.columns)), rotation=90, label=dataframe.columns)

    fig.set_figwidth(10)
    fig.set_figheight(24)
    
    ax2.set_ylim((lim2_x, lim2_y))
    ax3.set_ylim((lim3_x,lim3_y))
    ax5.set_ylim((lim5_x,lim5_y))

## In development

## to apply constraints from spreadsheet
constraint_dict = check_constraint_data_file(file_name =udef_constraint_data_file, 
                                             sheet_name = udef_constraint_sheet, 
                                             time_interval = udef_time_interval)

constraints_same_across_time_phases(cobra_model)

make_biomass_composition(cobra_model, udef_biomass_file, udef_biomass_composition, udef_biomass_sheet)
#already in core model: AraCore, Biomass
#in spreadsheet: de_Oliverira_DalMolin_et_al_AraGEM

phased_model, time_phase_dict = set_up_time_phases(cobra_model, time_interval = udef_time_interval)

changing_charge_states_in_vacuole_rxns(phased_model, file_name= "time-phase-constraints_test.xlsx", 
                                       constraint_sheet=udef_constraint_sheet, ph_sheet= "charge_states", time_interval =udef_time_interval)

creating_linker_fluxes(phased_model, constraint_data = constraint_dict, 
                       time_interval = udef_time_interval, protons=False)

light_dark_phases_constraints(phased_model, constraint_data=constraint_dict, 
                              manual = False, biomass_composition = udef_biomass_composition, maintenance_even_distribution = True)

apply_constraints_for_each_phase(phased_model, constraint_data= constraint_dict)

#########
constraint_dict = check_constraint_data_file(file_name =udef_constraint_data_file, 
                                             sheet_name = udef_constraint_sheet, 
                                             time_interval = udef_time_interval)

constraints_same_across_time_phases(cobra_model)

make_biomass_composition(cobra_model, udef_biomass_file, udef_biomass_composition, udef_biomass_sheet)
#already in core model: AraCore, Biomass
#in spreadsheet: de_Oliverira_DalMolin_et_al_AraGEM

phased_model, time_phase_dict = set_up_time_phases(cobra_model, time_interval = udef_time_interval)

changing_charge_states_in_vacuole_rxns(phased_model, file_name= "time-phase-constraints_test.xlsx", 
                                       constraint_sheet=udef_constraint_sheet, ph_sheet= "charge_states", time_interval =udef_time_interval)

creating_linker_fluxes(phased_model, constraint_data = constraint_dict, 
                       time_interval = udef_time_interval, protons=False)

light_dark_phases_constraints(phased_model, constraint_data=constraint_dict, 
                              manual = False, biomass_composition = udef_biomass_composition, maintenance_even_distribution = True)


apply_constraints_for_each_phase(phased_model, constraint_data= constraint_dict)

solution, model = optimise_model(phased_model, objective_reaction = "AraCore_Biomass_tx_light")

solution, model = optimise_model(phased_model, objective_reaction = "AraCore_Biomass_tx_light")

In [28]:
def save_fluxes_by_timephase(model, solution, phases, file_name):
    """Function to save the fluxes of multiple different solutions in a spreadsheet. 
    Should be fine if a reaction is not present in every solution."""
    
    import pandas as pd
    from datetime import date
    
    today = date.today()
    dated_file_name = str(today) + "_" + file_name 
    
    excel_file = pd.ExcelWriter(dated_file_name+'.xlsx', engine='xlsxwriter')
    
    
    DF1 = pd.DataFrame()
    
    
    rxns_list = []
    for rxn in model.reactions:
        if rxn.id[:-3] not in rxns_list:
            rxns_list.append(rxn.id[:-3])
            
    print(len(rxns_list))
        
    DF1['ID'] = rxns_list   
    
    for phase in phases:
        print(phase)
        solution_fluxes_list = []
        for rxn in model.reactions:
            if rxn.id[-2:] == phase:
                print(rxn.id)
                if rxn in solution.fluxes:
                    solution_fluxes_list.append(model.reactions.rxn.flux)
                    print(rxn)
#                 else:
#                     solution_fluxes_list.append("reaction not present in this solution")
#                     print(rxn, "not present in this solution")
#             else:
#                 solution_fluxes_list.append(0)
        print(len(solution_fluxes_list))
        DF1[phase] = solution_fluxes_list
        

    DF1.to_excel(excel_file, sheet_name='Fluxes')
    
    """counter = 0
    for solution in solutions_dict:
        idList = []
        solution_fluxes_list = []

        for x in solutions_dict[solution].fluxes.index:
            idList.append(x)
            solution_fluxes_list.append(solutions_dict[solution].fluxes[x])

        DF1[solution] = solution_fluxes_list
    DF1['ID'] = idList
    DF1_sorted = DF1.sort_values(by='solution_0', ascending=False)
    DF1.to_excel(excel_file, sheet_name='Fluxes')"""
    
    excel_file.save()
    
 

In [29]:
######### TRYING TO MAKE IT WRITE EVERYTHING IN CORRECT ORDER, even when <g is somewhere within a <marker


def svg_fluxmap_arrow_modification(previous_flux_map_file_name, csv_file_with_rxns_and_fluxes, col_of_rxns,
                                   col_of_fluxes, name_of_new_file, scaling = True, minimum_width = 0.1,
                                  maximum_width = 10, threshold_for_arrow = 5, opacity_for_zero_flux=0.5):
    """
    takes as input:
# may be outdated        - a .txt file of the .svg file containing the flux map with named arrows
        - a .csv file with the names of the reactions in one column and the fluxes in another
            - the name of the column with rxns
            - the name of the column with the corresponding fluxes
    creates as output:
        - a .svg file that looks like the input file except that the arrow width represents the fluxes
            the fluxes input file
    """
    
    try:
        previous_file_txt = previous_flux_map_file_name.replace(".svg", ".txt")
        shutil.copyfile(previous_flux_map_file_name, previous_file_txt)
        
        flux_map_file = open(previous_flux_map_file_name, "r")
        contents = flux_map_file.readlines()
        flux_map_file.close()
        print("Old .svg file read.")

        try:
            df = pd.read_csv(csv_file_with_rxns_and_fluxes)
            rxn_and_flux_dict = {}
            for index, elem in df.iterrows():
                rxn = elem[col_of_rxns]
                flux = elem[col_of_fluxes]
                rxn_and_flux_dict[rxn] = flux
                
            print("Spreadsheet read")
        except KeyError:
            print("Error: Check that the column names are correct.\n")
            raise

        if scaling:
            scaled_rxn_and_flux_dict = {}
            rxn_and_flux_dict_abs = {}
            print("The fluxes will be scaled according to the set minimum and maximum width values given.\n")
            max_flux = max(rxn_and_flux_dict.values())
            for ele in rxn_and_flux_dict: 
                rxn_and_flux_dict_abs[ele] = abs(rxn_and_flux_dict[ele])
            min_flux = min(rxn_and_flux_dict_abs.values())
            proportion_fluxes = (max_flux-min_flux)/100
            proportion_widths = (maximum_width-minimum_width)/100

            for rxn in rxn_and_flux_dict:
                if rxn_and_flux_dict[rxn] != 0 and round(rxn_and_flux_dict[rxn],threshold_for_arrow) > 0:
                    scaled_flux = (rxn_and_flux_dict[rxn] - min_flux)/proportion_fluxes *proportion_widths + minimum_width
                elif rxn_and_flux_dict[rxn] != 0 and round(rxn_and_flux_dict[rxn], threshold_for_arrow) < 0:
                    scaled_flux = (-1)*((abs(rxn_and_flux_dict[rxn]) - min_flux)/proportion_fluxes *proportion_widths + minimum_width)
                else:
                    scaled_flux = 0
                scaled_rxn_and_flux_dict[rxn] = scaled_flux
        else:
            print("Scaling was not used, the arrow width will correspond to the absolute value of the flux.\n")
        
        ### seems to be working with multi-layer .svg

        ## opens a new file, where new flux map will be created
        new_file = open(name_of_new_file, "w")
        
        if scaling:
            flux_dictionary = scaled_rxn_and_flux_dict
        else:
            flux_dictionary = rxn_and_flux_dict
        same_path = False
        path_string = ""
        for line in contents:
            if "<path" in line:
                path_string = ""
                same_path = True
                path_string += line
            elif same_path and ">" not in line:
                path_string += line
            elif same_path and ">" in line:
                path_string += line
                same_path = False
                
                for rxn in flux_dictionary:
                    rxn_id = "id=\"" +rxn+"\""
                    if rxn_id in path_string:
                        print("rxn_id:",rxn_id)
                        if flux_dictionary[rxn] < 0 and round(flux_dictionary[rxn],threshold_for_arrow) != 0:
                            print("Found arrow for reaction:",rxn_id, "Flux:", str(round(flux_dictionary[rxn],5)))
                            after_width = path_string.partition("stroke-width:")[2]
                            before_width = path_string.partition(after_width)[0]
                            width_str = after_width.partition(";")[0]
                            before_width_plus_old_width = before_width +width_str
                            before_width_plus_new_width = before_width + str(abs(flux_dictionary[rxn]))
                            new_width_string = path_string.replace(before_width_plus_old_width,before_width_plus_new_width)
                            
                            arrow_text = ";stroke-miterlimit:4;stroke-dasharray:none;marker-start:url(#TriangleInS)"
                            after_opacity = new_width_string.partition("stroke-opacity:1")[2]
                            before_opacity = new_width_string.partition(after_opacity)[0]
                            arrow_string = before_opacity + arrow_text + after_opacity
                            
                            path_string = arrow_string
                        
                        elif flux_dictionary[rxn] > 0 and round(flux_dictionary[rxn],threshold_for_arrow) != 0:
                            print("Found arrow for reaction:",rxn_id, "Flux:", str(round(flux_dictionary[rxn],5)))
                            after_width = path_string.partition("stroke-width:")[2]
                            before_width = path_string.partition(after_width)[0]
                            width_str = after_width.partition(";")[0]
                            before_width_plus_old_width = before_width +width_str
                            before_width_plus_new_width = before_width + str(flux_dictionary[rxn])
                            new_width_string = path_string.replace(before_width_plus_old_width,before_width_plus_new_width)
                            
                            arrow_text = ";stroke-miterlimit:4;stroke-dasharray:none;marker-end:url(#TriangleOutS)"
                            after_opacity = new_width_string.partition("stroke-opacity:1")[2]
                            before_opacity = new_width_string.partition(after_opacity)[0]
                            arrow_string = before_opacity+ arrow_text + after_opacity
                            
                            path_string = arrow_string
                        elif round(flux_dictionary[rxn],threshold_for_arrow) == 0:
                            print("Found arrow for reaction:",rxn_id, "but flux equals", str(round(flux_dictionary[rxn],threshold_for_arrow)))
                            after_opacity = path_string.partition("stroke-opacity:1")[2]
                            before_opacity = path_string.partition(after_opacity)[0]
                            opacity_string = before_opacity + ";opacity:"+str(opacity_for_zero_flux) + after_opacity
                            
                            path_string = opacity_string
                        else:
                            print("reaction fulfilled none of the conditions")


                new_file.write(path_string)
                print("-----path added to file-----")
                rxn_id = ""
                path_string = ""
            else:
                new_file.write(line)


        new_file.close()
        
    except FileNotFoundError:
        print("One of the input files specified does not exist.")
    
    return scaled_rxn_and_flux_dict ,rxn_and_flux_dict, rxn_and_flux_dict_abs, max_flux, min_flux

 

In [30]:
# save fluxes as .xslx
def save_fluxes(models, solutions_dict, file_name, query):
    """Function to save the fluxes of multiple different solutions in a spreadsheet. 
    Should be fine if a reaction is not present in every solution."""
    
    import pandas as pd
    from datetime import date
    
    today = date.today()
    dated_file_name = str(today) + "_" + file_name 
    
    excel_file = pd.ExcelWriter(dated_file_name+'.xlsx', engine='xlsxwriter')
    
    solution_fluxes_dict = {}
    
    DF1 = pd.DataFrame()
    
    
    rxns_id_list = []
    rxns_list = []
    for model in models:
        for rxn in model.reactions:
            if rxn.id not in rxns_id_list:
                rxns_id_list.append(rxn.id)
                rxns_list.append(rxn)
                rxns_id_list.append(str(rxn.id)+".upper_bound")
                rxns_list.append(rxn.upper_bound)
                rxns_id_list.append(str(rxn.id)+".lower_bound")
                rxns_list.append(rxn.lower_bound)
    
        
    DF1['ID'] = rxns_id_list
    DF1['Reaction'] = rxns_list
   
    
    for solution in solutions_dict:
        solution_fluxes_list = []
        for rxn in rxns_id_list:
            if rxn in solutions_dict[solution].fluxes:
                solution_fluxes_list.append(solutions_dict[solution].fluxes.get(rxn))
            else:
                solution_fluxes_list.append("reaction not present in this solution")
                print(rxn, "not present in this solution")
                                                
        DF1[solution] = solution_fluxes_list
    
    DF1.to_excel(excel_file, sheet_name='Fluxes')
    
    
    excel_file.save()

In [31]:
def flux_diff(model1, model2, process, rounding_nr=4):
    model1_flux = model1.reactions.get_by_id(process).flux
    model2_flux = model2.reactions.get_by_id(process).flux
    diff = round(model1_flux - model2_flux, rounding_nr)
    ratio = round(model1_flux / model2_flux, rounding_nr)
    perc_diff = round(diff/model2_flux, rounding_nr) * 100
    
    print("1", "\t\t", "2", "\t\t", "difference", "\t", "ratio", "\t\t", "% difference")
    print(round(model1_flux, rounding_nr), "\t", round(model2_flux, rounding_nr), "\t", diff, "\t", ratio, "\t", perc_diff)
    
    return diff, ratio, perc_diff

In [9]:
def normalise_by_obj_flux(model, objective_reaction):
    norm_model = model.copy()
    
    obj_flux = model.reactions.get_by_id(objective_reaction).flux
    
    for rxn in model.reactions:
        current_flux = rxn.flux
        norm_model.reactions.get_by_id(rxn.id).bounds = (current_flux/obj_flux, current_flux/obj_flux)
        
    solution = norm_model.optimize()
    
    return norm_model